In [1]:
# import libraries
from glob import glob
import os
import pandas as pd
import numpy as np
import re

# Output table/file names and template
OH_SWITCHES_TABLE ='IN_OH_SW.xlsx'
UG_SWITCHES_TABLE ='IN_UG_SW.xls'
OH_TX_TABLE = 'IN_OH_TX.xlsx'
UG_TX_TABLE = 'IN_UG_TX.xlsx'
POLES_TABLE = 'IN_POLES.xlsx'
UG_PRI_CABLE_TABLE = 'IN_CABLES.xlsx'
NTWK_TX_TABLE = 'IN_NTWK_TX.xlsx'
OH_SWITCHES_TABLE_TEMPLATE ='IN_OH_SW_TEMPLATE.xlsx'
UG_SWITCHES_TABLE_TEMPLATE ='IN_UG_SW_TEMPLATE.xls'
OH_TX_TABLE_TEMPLATE = 'IN_OH_TX_TEMPLATE.xlsx'
UG_TX_TABLE_TEMPLATE = 'IN_UG_TX_TEMPLATE.xlsx'
POLES_TABLE_TEMPLATE = 'IN_POLES_TEMPLATE.xlsx'
UG_PRI_CABLE_TABLE_TEMPLATE = 'IN_CABLES_TEMPLATE.xlsx'
NTWK_TX_TABLE_TEMPLATE = 'IN_NTWK_TX_TEMPLATE.xlsx'

# asset_class_code(ACC) names
OH_SWITCHES_ASSET_CLASS ='OH_SWITCH'
UG_SWITCHES_ASSET_CLASS ='UG_SWITCH'
OH_TX_ASSET_CLASS = 'OH_TX'
UG_TX_ASSET_CLASS = 'UG_TX'
POLES_ASSET_CLASS = 'POLE'
UG_PRI_CABLE_ASSET_CLASS = 'UG_CABLE'
NTWK_TX_ASSET_CLASS = 'NTWK_TX'

ASSET_CLASS ='asset_class_code'
ASSET_SUBCLASS ='asset_subclass_code'

#Template folder
ASSET_TEMPLATE_FOLDER='AssetDataTemplates'

#******************************************************
# Declare column names
#******************************************************
UG_SWITCHES_COLS =['asset_id','id','asset_subclass_code','asset_class_code','install_year','hi',
                   'phasing','prid','circuit','tx_phase','in_valley','tie_feeder']

OH_TX_COLS = ['asset_id','asset_class_code','id','circuit','install_year','asset_subclass_code','hi',
              'phasing','primary_voltage','kva','tx_residential','tx_commercial','tx_industrial','device_residential',
              'device_commercial','device_industrial','upstream_device','prid','in_valley','pcb','banking']

UG_TX_COLS = ['asset_id','asset_subclass_code','asset_class_code','install_year','hi','phasing','prid','circuit',
              'primary_voltage','kva','in_valley','tx_residential','tx_commercial','tx_industrial','device_residential',
              'device_commercial','device_industrial','upstream_device','pcb','pedestal','switchable','switch_type','id']

UG_PRI_CABLE_COLS = ['asset_id','id','install_year','hi','asset_subclass_code','asset_class_code','phasing','prid',
                     'circuit','arrangement','installation','material','cable_size','config','length','num_splices',
                     'num_cables','prid_residential','prid_commercial','prid_industrial','nominal_voltage',
                     'wc_prid_catastrophic_res','wc_prid_catastrophic_comm','wc_prid_catastrophic_ind','cable_phase',
                     'wc_replacement','wc_switching_res','wc_switching_comm','wc_switching_ind','wc_switching_duration']

POLES_COLS = ['asset_id','asset_class_code','asset_subclass_code','install_year','hi character','phasing character',
              'prid character','pole_class','tx','tx_type','circuit1','circuit2','circuit3','circuit4','in_valley',
              'tx_residential','tx_commercial','tx_industrial','height','num_circuits','device','tx_kva','id','prid2',
              'prid3','prid4','tx_pcb']

NTWK_TX_COLS = ['asset_id','asset_subclass_code','asset_class_code','install_year','hi','phasing','prid','circuit',
                'primary_voltage','kva character','load double','id character','network_type','tx1','tx2','tx3','tx4']

#******************************************************
# fileName - iterate through entire folder :)
fileName = 'Original_FiveAssetClasses.xlsx'
file_PolesLatLong = 'Asset_XY_files/V2_LatLongPoles.xls'
file_SwitchesLatLong = 'Asset_XY_files/V2_LatLongSwitches.xls'
file_PriOHLatLong = 'Asset_XY_files/V2_LatLongPriOH.xls'
file_PriUGLatLong = 'Asset_XY_files/V2_LatLongPriUG.xls'
file_TxLatLong = 'Asset_XY_files/V2_LatLongTx.xls'
#fileNameOtherDevices = 'Other Device Numbers.xls'

# Read xlsx file into dataframes
with pd.ExcelFile(fileName) as xlsx:
    #dfTopology = pd.read_excel(xlsx, 'Topology', index_col=None, na_values=['NA']) # IGNORE for now
    dfTransformersV1 = pd.read_excel(xlsx, 'Transformers') # 280 rows
    dfSwitchesV1 = pd.read_excel(xlsx, 'Switches') # Tot:239 - R/Y/B: 116/108/103 values; based on phases
    dfPolesV1 = pd.read_excel(xlsx, 'Poles') # 239 rows; 'Spot Number\n' col contains unique tx ids
    dfCablesV1 = pd.read_excel(xlsx, 'UGPrimaryCables')
    dfFusesV1 = pd.read_excel(xlsx, 'Fuses') # 44 items
    dfUGStructuresV1 = pd.read_excel(xlsx,'UGStructures')

# Read xlsx file into dataframes
with pd.ExcelFile(file_PolesLatLong) as xls:
    dfPolesLatLongV1 = pd.read_excel(xls, 'Sheet1')
    
with pd.ExcelFile(file_SwitchesLatLong) as xls:
    dfSwitchesLatLongV1 = pd.read_excel(xls, 'Sheet1')

with pd.ExcelFile(file_PriOHLatLong) as xls:
    dfOHCondLatLongV1 = pd.read_excel(xls, 'Sheet1')

with pd.ExcelFile(file_PriUGLatLong) as xls:
    dfCablesLatLongV1 = pd.read_excel(xls, 'Sheet1')
    
with pd.ExcelFile(file_TxLatLong) as xls:
    dfTxLatLongV1 = pd.read_excel(xls, 'Sheet1')
    
Summary = {'Transformers:': dfTransformersV1.shape, 'Switches:': dfSwitchesV1.shape,'Poles:': dfPolesV1.shape, 
           'Cables:': dfCablesV1.shape, 'Fuses:':dfFusesV1.shape, 'UGStructures:':dfUGStructuresV1.shape}
dfSummary = pd.DataFrame(Summary)

# Make one copy
dfTransformersV2 = dfTransformersV1
dfSwitchesV2 = dfSwitchesV1
dfPolesV2 = dfPolesV1
dfCablesV2 = dfCablesV1
dfFusesV2 = dfFusesV1
dfUGStructuresV2 = dfUGStructuresV1

# 17 columns dropped
dropCommonColumns = ['OBJECTID','WORKORDERID','FIELDVERIFY','COMMENTS','CREATIONUSER','DATECREATED','LASTUSER',
                     'DATEMODIFIED','WORKREQUESTID','DESIGNID','WORKLOCATIONID','WMSID','WORKFLOWSTATUS',
                     'WORKFUNCTION','GISONUMBER','GISOTYPENBR','OWNERSHIP']

#******************************************************
# FUNCTIONS
#******************************************************
def drop_columns(dfAssetClass, dropColumns):
    dfAssetClass = dfAssetClass.drop(dropColumns, axis=1)
    return dfAssetClass

def new_columns(dfAssetClass, numAssetRows, columnID):
    dfAssetClass[columnID] = pd.DataFrame(np.empty([numAssetRows,1]).cumsum(axis=1))
    dfAssetClass.loc[:,columnID] = np.nan
    return dfAssetClass[columnID]

#******************************************************
#Drop all common columns 
#******************************************************
dfSwitchesV2 = drop_columns(dfSwitchesV2, dropCommonColumns)
dfTransformersV2 = drop_columns(dfTransformersV2, dropCommonColumns)
dfFusesV2 = drop_columns(dfFusesV2,dropCommonColumns)
dfCablesV2 = drop_columns(dfCablesV2, dropCommonColumns)
dfUGStructuresV2 = drop_columns(dfUGStructuresV2, dropCommonColumns)
dfPolesV2 = drop_columns(dfPolesV2, dropCommonColumns)

dfSwitchesV2['FEEDERID'] = dfSwitchesV2['FEEDERID'].astype(str)
dfTransformersV2['FEEDERID'] = dfTransformersV2['FEEDERID'].astype(str)
dfFusesV2['FEEDERID'] = dfFusesV2['FEEDERID'].astype(str)
dfCablesV2['FEEDERID'] = dfCablesV2['FEEDERID'].astype(str)

dfSwitchesV2['FEEDERID'] = dfSwitchesV2['FEEDERID'].apply(lambda x: re.sub('[\s+]', '', x))
dfTransformersV2['FEEDERID'] = dfTransformersV2['FEEDERID'].apply(lambda x: re.sub('[\s+]', '', x))
dfFusesV2['FEEDERID'] = dfFusesV2['FEEDERID'].apply(lambda x: re.sub('[\s+]', '', x))
dfCablesV2['FEEDERID'] = dfCablesV2['FEEDERID'].apply(lambda x: re.sub('[\s+]', '', x))

# Make one copy
dfTransformers = dfTransformersV2
dfSwitches = dfSwitchesV2
dfPoles = dfPolesV2
dfCables = dfCablesV2
dfFuses = dfFusesV2
dfUGStructures = dfUGStructuresV2

SummaryV2 = {'Transformers:': dfTransformers.shape, 'Switches:': dfSwitches.shape,'Poles:': dfPoles.shape, 
           'Cables:': dfCables.shape, 'Fuses:':dfFuses.shape, 'UGStructures:':dfUGStructures.shape}
dfSummaryV2 = pd.DataFrame(SummaryV2)
#print(dfSummary)
print(dfSummaryV2)   

   Cables:  Fuses:  Poles:  Switches:  Transformers:  UGStructures:
0     3865     736   18961        537           3618          16883
1       22      29      14         35             40             23


In [2]:
#*****************************************************************************************************
# Cell # 1.2 - Nearest Neighbor algorithm
#*****************************************************************************************************
from sklearn.neighbors import KNeighborsClassifier
import random
from numpy.random import permutation
import math
#nearest_neighbor(df_filled, 'x','y','OH_FEEDERID',3,df_empty)
def nearest_neighbor(dfMain, trainX, trainY, classX, neighborCount, dfUnknown):
    #https://www.dataquest.io/blog/k-nearest-neighbors/
    # Randomly shuffle the index of df_filled.
    random_indices = permutation(dfMain.index)
    # Set a cutoff for how many items we want in the test set (in this case 1/3 of the items)
    test_cutoff = math.floor(len(dfMain)/3)
    # Generate the test set by taking the first 1/3 of the randomly shuffled indices.
    dfMain_test = dfMain.loc[random_indices[1:test_cutoff]]
    # Generate the train set with the rest of the data.
    dfMain_train = dfMain.loc[random_indices[test_cutoff:]]

    for k in [1, 2, 3, 5, 10, 20]:
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(dfMain_train[[trainX, trainY]], dfMain_train[classX])

        predictions = knn.predict(dfMain_test[[trainX,trainY]])
        prediction_results = dfMain_test[classX] == predictions
        print('With k =  ',k,',a score of: ', prediction_results.mean()*100)

    # Let's initialize a classifier
    knn = KNeighborsClassifier(n_neighbors=neighborCount)
    # knn.fit takes two parameters # First, the content we want to train on. For us it's height and weight.
    # Secondly, how we're classifying each element of the training data. We're classifying by position!
    knn.fit(dfMain_train[[trainX, trainY]], dfMain_train[classX])
    predictions = knn.predict(dfMain_test[[trainX,trainY]])
    prediction_results = dfMain_test[classX] == predictions
    print('Prediction accuracy: ',prediction_results.mean()*100) 
    # same as mse = (((prediction_results) ** 2).sum()) / len(predictions)
    predictedValues = knn.predict(dfUnknown[[trainX, trainY]])
    return predictedValues

# Split the df_filled to train and test data
#from sklearn.cross_validation import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X_wine, y_wine,test_size=0.30, random_state=123)

# Compute the mean squared error of our predictions.
# mse = (((predictions - actual) ** 2).sum()) / len(predictions)

C:\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [3]:
#*****************************************************************************************************
# Cell # 2: Switches and Transformers
#*****************************************************************************************************

# Save future wait times while running
dfTransformers = dfTransformersV2
dfSwitches = dfSwitchesV2
dfPoles = dfPolesV2
# dfCables = dfCablesV2
# dfFuses = dfFusesV2
# dfUGStructures = dfUGStructuresV2

# Lat long df
dfPolesLatLong = dfPolesLatLongV1
dfSwitchesLatLong = dfSwitchesLatLongV1
dfOHCondLatLong = dfOHCondLatLongV1
dfCablesLatLong = dfCablesLatLongV1

#******************************************************
# ASSET CLASS SPECIFIC DICTIONARIES
#******************************************************
# OPERATING VOLTAGE 190=8kv, 250=13.8kv, 1267 = 0kv, 1237 = 138kv
# Assets: Transformers,
operatingVoltageDict = {'190':'8000','250':'13800','1267':'0','1237':'138000'}

# Phasing change - need to change it to 'str' type, int/float dict key lookup doesn't work
# Assets: UG Switches, Transformers
#phasingDict = {'1.0': '1Ph', '2.0':'1Ph','4.0':'1Ph','3.0':'2Ph','5.0':'2Ph','6.0':'2Ph','7.0':'3Ph'}
phasingDict = {'1': '1', '2':'1','4':'1','3':'2','5':'2','6':'2','7':'3'}

# UG Switches
dictSGassetSubclass = {'PMH-3':'AIR_INSULATED_LIVEFRONT_PMH-9','PMH-5':'AIR_INSULATED_LIVEFRONT_PMH-9',
                       'PMH-9':'AIR_INSULATED_LIVEFRONT_PMH-9','PMH-11':'AIR_INSULATED_LIVEFRONT_PMH-11',
                       'PME-9':'AIR_INSULATED_DEADFRONT_PME-9','PME-10':'AIR_INSULATED_DEADFRONT_PME-9',
                       'PME-11':'AIR_INSULATED_DEADFRONT_PME-11','VISTA-321':'SF6_INSULATED_SWITCH_VISTA-321',
                       'VISTA-422':'SF6_INSULATED_SWITCH_VISTA-422','VISTA-431':'SF6_INSULATED_SWITCH_VISTA-431',
                       '422':'SC_ELEC','431':'SC_ELEC','321':'SC_ELEC','G&W':'GW',
                       'NET':'CARTE_ELEC_LTD'}

# Transformers
# dictOHTxSubclass = {'1':'Standard 1Ph','9':'Standard 3Ph','10':'Standard 2Ph'}
# dictUGTxSubclass = {'2':'Padmount 1Ph','3':'Network Submersible','5':'Submersible', '7':'Padmount 3Ph'}
dictOHTxSubclass = {'1':'POLE_TOP','9':'POLE_TOP','10':'POLE_TOP'}
dictUGTxSubclass = {'2':'PAD_MOUNTED','3':'NETWORK_SUBMERSIBLE','5':'SUBMERSIBLE', '7':'PAD_MOUNTED'}

#****************************************************************************
# UG Switches - Reading SwitchGears
#****************************************************************************
#**************************
# Reading SwitchGears
#***************************
fileNameOtherDevices = 'Other Device Numbers.xlsx'
# Read Other Device Numbers into dataframes
with pd.ExcelFile(fileNameOtherDevices) as xls:
    dfSwitchGears = pd.read_excel(xls, 'SWITCHGEARS') # 280 rows

dropSGcols = ['Switch Gear', 'Adrs #','Location','City','Notes','To Type','Inst. Date','Mftr.','Catalog#','Serial#',
             'DOM','Comments']

dfSwitchGears = drop_columns(dfSwitchGears, dropSGcols)
#dfSwitchGears = dfSwitchGears.dropna() # drop all rows with NaN values

# 'Type' -> 'PMH'
# 'Loc_No' -> '149-S'
dfSwitchGears['Type'] = dfSwitchGears['Type'].fillna(method='ffill')
numSGrows = len(dfSwitchGears['Loc_No'])
dfSwitchGears['ASSET_SUBCLASS'] = new_columns(dfSwitchGears, numSGrows, 'ASSET_SUBCLASS')
dfSwitchGears =dfSwitchGears.astype(str)
#dfSwitchGears['Loc_No'] = dfSwitchGears.iloc[:,'Loc_No'].apply[s.lstrip("0") for s in listOfNum]
dfSwitchGears['Loc_No'] = [s.lstrip("0") for s in dfSwitchGears['Loc_No']]
dfSwitchGears['ASSET_SUBCLASS'] = dfSwitchGears['Type'].apply(lambda x: dictSGassetSubclass[x])

#*******************
# Drop columns
#*******************
dropSwitchesCols = ['ANCILLARYROLE','ENABLED','FEEDERINFO','ELECTRICTRACEWEIGHT','LOCATIONID','GPSDATE','LABELTEXT',
                    'OPERATINGVOLTAGE', 'NOMINALVOLTAGE', 'MAXOPERATINGVOLTAGE','MAXCONTINUOUSCURRENT','PRESENTPOSITION_R', 
                    'PRESENTPOSITION_Y', 'PRESENTPOSITION_B','NORMALPOSITION_R','NORMALPOSITION_Y','NORMALPOSITION_B', 
                    'SCADACONTROLID', 'SCADAMONITORID','PREFERREDCIRCUITSOURCE','TIESWITCHINDICATOR',
                    'GANGOPERATED', 'MANUALLYOPERATED','FEATURE_STATUS','HYPERLINK','HYPERLINK_PGDB','SYMBOLROTATION',
                    'INSULATOR_MATERIAL']

#******************************************************
# drop asset columns
#******************************************************
dfSwitches = drop_columns(dfSwitches,dropSwitchesCols)
#******************************************************
# FILTER OUT ASSET CLASSES WITH THEIR RESPECTIVE SUBTYPES
#******************************************************
# To avoid index vs copy error: pd.DataFrame...necessary (spent 4 hours getting rid of the warning error!)
# UG Switches rows
dfSwitches = dfSwitches[dfSwitches.SUBTYPECD == 6]
numSwitchRows = len(dfSwitches['DEVICENUMBER'])

#*******************
# ADD ADDITIONAL COLUMNS AND FILL WITH NaNs
#*******************
# UG Switches
dfSwitches['HI'] = new_columns(dfSwitches,numSwitchRows, 'HI')
dfSwitches['TX_PHASE'] = new_columns(dfSwitches,numSwitchRows, 'TX_PHASE')
dfSwitches['IN_VALLEY'] = new_columns(dfSwitches,numSwitchRows, 'IN_VALLEY')
dfSwitches['PRID'] = new_columns(dfSwitches,numSwitchRows, 'PRID')
#**************************
# RENAME ASSET COLUMNS
#**************************
# Rename Switch columns
dfSwitches = dfSwitches.rename(columns={'SUBTYPECD':ASSET_CLASS,
                                        'DEVICENUMBER':'ID',
                                        'COMPATIBLEUNITID':ASSET_SUBCLASS,
                                        'PHASEDESIGNATION':'PHASING',
                                        'FEEDERID':'CIRCUIT', 
                                        'FEEDERID2':'TIE_FEEDER',
                                        'INSTALLATIONDATE':'INSTALL_YEAR'})
# Separate year
dfSwitches['INSTALL_YEAR'] = dfSwitches['INSTALL_YEAR'].apply(lambda x: x.year)

# Replace 'Asset Subclass' col with actual names
dfSwitches['ID'] = dfSwitches['ID'].astype(str)
dfSwitches=pd.merge(dfSwitches, dfSwitchGears, how='left', left_on='ID', right_on='Loc_No')
#dfSwitches['ID'] = dfSwitchGears['Loc_No'].apply(lambda x: )
#df.merge(df1, on='sku', how='left')
# print(len(pd.unique(dfSwitchGears['Loc_No'].values.ravel()))) # 111

#******************************************************
switchesDropMoreCols = [ASSET_SUBCLASS, 'Loc_No']
dfSwitches = dfSwitches.drop(switchesDropMoreCols, axis=1)
dfSwitches = dfSwitches.rename(columns={'ASSET_SUBCLASS': ASSET_SUBCLASS})

#******************************************************
# Rename proper asset nomenclature
dfSwitches[ASSET_CLASS] = UG_SWITCHES_ASSET_CLASS

#******************************************************
# Rename proper asset nomenclature
dfSwitches[ASSET_CLASS] = UG_SWITCHES_ASSET_CLASS

#******************************************************
#phasingDict = {'1.0': '1Ph', '2.0':'1Ph','4.0':'1Ph','3.0':'2Ph','5.0':'2Ph','6.0':'2Ph','7.0':'3Ph'}
# UG Switches - 'phasing' col
dfSwitches['PHASING'] = dfSwitches['PHASING'].astype(int)
dfSwitches['PHASING'] = dfSwitches['PHASING'].astype(str)
dfSwitches['PHASING'] = dfSwitches['PHASING'].apply(lambda x: phasingDict[x])

#******************************************************
# Lower case column names
dfSwitches.columns = map(str.lower, dfSwitches.columns)

#******************************************************
# REARRANGE COLUMNS
#******************************************************
# *All tables need 'asset_id' - rename index
# UG Switches
# UG_SWITCHES_COLS =['asset_id','id','asset_subclass_code','asset_class_code','install_year','hi',
#                    'phasing','prid','circuit','tx_phase','in_valley','tie_feeder']
dfSwitches =dfSwitches[['id','asset_subclass_code','asset_class_code','install_year','hi','phasing','prid','circuit','tx_phase','in_valley','tie_feeder','type']]

#*********************
# OUTPUT DATAFRAMES TO EXCEL FILES
#*********************
# Output table/file names
# UG SWITCH
dfUGSwLater = dfSwitches
MasterFile = pd.ExcelWriter(UG_SWITCHES_TABLE_TEMPLATE)
dfSwitches.to_excel(MasterFile, 'Sheet1')
MasterFile.save()

#********************
# Drop columns
#*******************

dropTxCols = ['ANCILLARYROLE', 'ENABLED', 'FEEDERID2', 'FEEDERINFO', 'ELECTRICTRACEWEIGHT', 'LOCATIONID', 'SYMBOLROTATION', 
              'GPSDATE', 'LABELTEXT', 'NOMINALVOLTAGE', 'GROUNDREACTANCE', 'GROUNDRESISTANCE', 
              'MAGNETIZINGREACTANCE', 'MAGNETIZINGRESISTANCE', 'HIGHSIDEGROUNDREACTANCE','HIGHSIDEGROUNDRESISTANCE', 
              'HIGHSIDEPROTECTION', 'LOCATIONTYPE','COOLINGTYPE', 'FEATURE_STATUS','KVA', 'DEMAND_KVA',
              'DEMAND_DATE_MM_DD_YYYY', 'STREET_LIGHT_FACILITY', 'HIGHSIDECONFIGURATION', 'LOWSIDECONFIGURATION',
              'LOWSIDEGROUNDRESISTANCE', 'LOWSIDEVOLTAGE', 'LATITUDE', 'LONGITUDE']

#******************************************************
# drop asset columns
#******************************************************
dfTransformers = drop_columns(dfTransformers,dropTxCols)

#******************************************************
# FILTER OUT ASSET CLASSES WITH THEIR RESPECTIVE SUBTYPES
#******************************************************
# To avoid index vs copy error: pd.DataFrame...necessary (spent 4 hours getting rid of the warning error!)
# number of rows/observations 
numTxRows = len(dfTransformers['DEVICENUMBER'])

#******************************************************
# RENAME ASSET COLUMNS
#******************************************************

# Rename Transformer columns
dfTransformers = dfTransformers.rename(columns={'DEVICENUMBER':'ID',
                                                'PHASEDESIGNATION':'Type',
                                                'INSTALLATIONDATE':'INSTALL_YEAR',
                                                'FEEDERID':'CIRCUIT',
                                                'RATEDKVA':'KVA'})

# Separate year
dfTransformers['INSTALL_YEAR'] = dfTransformers['INSTALL_YEAR'].apply(lambda x: x.year)

#******************************************************
# ADD ADDITIONAL COLUMNS AND FILL WITH NaNs
#******************************************************

# Transformers
dfTransformers[ASSET_CLASS] = new_columns(dfTransformers, numTxRows, ASSET_CLASS)
dfTransformers['HI'] = new_columns(dfTransformers, numTxRows,'HI')
dfTransformers['PRID'] = new_columns(dfTransformers, numTxRows,'PRID')
dfTransformers['IN_VALLEY'] = new_columns(dfTransformers, numTxRows,'IN_VALLEY')
dfTransformers['TX_RESIDENTIAL'] = new_columns(dfTransformers, numTxRows,'TX_RESIDENTIAL')
dfTransformers['TX_COMMERCIAL'] = new_columns(dfTransformers, numTxRows,'TX_COMMERCIAL')
dfTransformers['TX_INDUSTRIAL'] = new_columns(dfTransformers, numTxRows,'TX_INDUSTRIAL')
dfTransformers['DEVICE_RESIDENTIAL'] = new_columns(dfTransformers, numTxRows,'DEVICE_RESIDENTIAL')
dfTransformers['DEVICE_COMMERCIAL'] = new_columns(dfTransformers, numTxRows,'DEVICE_COMMERCIAL')
dfTransformers['DEVICE_INDUSTRIAL'] = new_columns(dfTransformers, numTxRows,'DEVICE_INDUSTRIAL')
dfTransformers['UPSTREAM_DEVICE'] = new_columns(dfTransformers, numTxRows,'UPSTREAM_DEVICE')
dfTransformers['PCB'] = new_columns(dfTransformers, numTxRows,'PCB')

#******************************************************
# FILTER OUT ASSET CLASSES WITH THEIR RESPECTIVE SUBTYPES
#******************************************************
# To avoid index vs copy error: pd.DataFrame...necessary (spent 4 hours getting rid of the warning error!)
# UG Tx: 2/3/5/7 - 1Ph/Ntwk/Sub/Pad 3Ph [1436,27,4,507: 1642 counts]
dfUGTransformers = pd.DataFrame(dfTransformers[(dfTransformers.SUBTYPECD == 2) | 
                                  (dfTransformers.SUBTYPECD == 3) | 
                                  (dfTransformers.SUBTYPECD == 5) | 
                                  (dfTransformers.SUBTYPECD == 7) ])

# OH Tx: 1/9/10 - 1Ph/3Ph/2Ph [1125/510/7: 1347 counts]
dfOHTransformers = pd.DataFrame(dfTransformers[(dfTransformers.SUBTYPECD == 1) | 
                                  (dfTransformers.SUBTYPECD == 9) | 
                                  (dfTransformers.SUBTYPECD == 10)])

#******************************************************
# Replace Asset class and 'SUBTYPECD' with actual tx types
#******************************************************
numOHTxRows = len(dfOHTransformers['ID'])
numUGTxRows = len(dfUGTransformers['ID'])

#******************************************************
# TRANSFORMERS
#******************************************************
dfOHTransformers['SUBTYPECD'] = dfOHTransformers['SUBTYPECD'].astype(str)
dfUGTransformers['SUBTYPECD'] = dfUGTransformers['SUBTYPECD'].astype(str)

#Try using .loc[row_indexer,col_indexer] = value instead
dfOHTransformers.loc[:,'SUBTYPECD'] = dfOHTransformers['SUBTYPECD'].apply(lambda x: dictOHTxSubclass[x])
dfUGTransformers.loc[:,'SUBTYPECD'] = dfUGTransformers['SUBTYPECD'].apply(lambda x: dictUGTxSubclass[x])

# Fill in Asset and asset subclass columns
dfOHTransformers = dfOHTransformers.rename(columns={'SUBTYPECD':ASSET_SUBCLASS})
dfUGTransformers = dfUGTransformers.rename(columns={'SUBTYPECD':ASSET_SUBCLASS})

# Remaining OH Tx and UG Tx specific columns
dfOHTransformers['BANKING'] = new_columns(dfOHTransformers, numOHTxRows,'BANKING')
dfOHTransformers['BANKING'] = dfOHTransformers['UNITS'].apply(lambda x: x)
dfUGTransformers['BANKING'] = new_columns(dfUGTransformers, numOHTxRows,'BANKING')
dfUGTransformers['BANKING'] = dfUGTransformers['UNITS'].apply(lambda x: x)
dfUGTransformers['PEDESTAL'] = new_columns(dfUGTransformers, numUGTxRows,'PEDESTAL')
dfUGTransformers['SWITCHABLE'] = new_columns(dfUGTransformers, numUGTxRows,'SWITCHABLE')
dfUGTransformers['SWITCH_TYPE'] = new_columns(dfUGTransformers, numUGTxRows,'SWITCH_TYPE')
#print(numOHTxRows, numUGTxRows)
#print(dfOHTransformers.columns)

# Tx Domain code tables
fileNameDomainCodes_Tx = 'DomainCodes_Tx.xlsx'
# Read Other Device Numbers into dataframes
with pd.ExcelFile(fileNameDomainCodes_Tx) as xls:
    #dfTopology = pd.read_excel(xlsx, 'Topology', index_col=None, na_values=['NA']) # IGNORE for now
    dfUGTxDomainCodes = pd.read_excel(xls, 'UGTransformers')
    dfOHTxDomainCodes = pd.read_excel(xls, 'OHTransformers')

# Convert to string for merge purposes
dfOHTransformers['COMPATIBLEUNITID'] = dfOHTransformers['COMPATIBLEUNITID'].astype(str)
dfUGTransformers['COMPATIBLEUNITID'] = dfUGTransformers['COMPATIBLEUNITID'].astype(str)
dfOHTxDomainCodes['COMPATIBLEUNITID'] = dfOHTxDomainCodes['COMPATIBLEUNITID'].astype(str)
dfUGTxDomainCodes['COMPATIBLEUNITID'] = dfUGTxDomainCodes['COMPATIBLEUNITID'].astype(str)
#print(dfUGTxDomainCodes.head())

#dfOHTransformers=pd.merge(dfOHTransformers, dfOHTxDomainCodes, how='left', on='COMPATIBLEUNITID')
#dfUGTransformers=pd.merge(dfUGTransformers, dfUGTxDomainCodes, how='left', on='COMPATIBLEUNITID')
dfOHTransformers=dfOHTransformers.merge(dfOHTxDomainCodes, how='left', on='COMPATIBLEUNITID')
dfUGTransformers=dfUGTransformers.merge(dfUGTxDomainCodes, how='left', on='COMPATIBLEUNITID')
#print(dfUGTransformers.head(2))

dropOHTxCols = ['COMPATIBLEUNITID','Description','PRIMARY_VOLTAGE','NAMEPLATE','PHASING','FAULTINDICATOR','UNITS','Tx_type_counts']
dropUGTxCols = ['COMPATIBLEUNITID','Description','PRIMARY_VOLTAGE','NAMEPLATE','PHASING','FAULTINDICATOR','UNITS','Tx_type_counts']
#'Fused','UNITS',
# drop columns
dfOHTransformers = drop_columns(dfOHTransformers,dropOHTxCols)
dfUGTransformers = drop_columns(dfUGTransformers,dropUGTxCols)

#******************************************************
# 3
#******************************************************
# Replace 'Asset Subclass' col with actual names
# Rename proper asset nomenclature
#dfSwitches[ASSET_CLASS] = OH_SWITCHES_ASSET_CLASS
dfOHTransformers[ASSET_CLASS] = OH_TX_ASSET_CLASS
dfUGTransformers[ASSET_CLASS] = UG_TX_ASSET_CLASS

#******************************************************
#phasingDict = {'1.0': '1Ph', '2.0':'1Ph','4.0':'1Ph','3.0':'2Ph','5.0':'2Ph','6.0':'2Ph','7.0':'3Ph'}

# OH and UG Tx - 'operational voltage'
#operatingVoltageDict = {'190':'8000','250':'13800','1267':'0','1237':'138000'}
dfOHTransformers['OPERATINGVOLTAGE'] = dfOHTransformers['OPERATINGVOLTAGE'].astype(str)
dfUGTransformers['OPERATINGVOLTAGE'] = dfUGTransformers['OPERATINGVOLTAGE'].astype(str)
dfOHTransformers['OPERATINGVOLTAGE'] = dfOHTransformers['OPERATINGVOLTAGE'].apply(lambda x: operatingVoltageDict[x])
dfUGTransformers['OPERATINGVOLTAGE'] = dfUGTransformers['OPERATINGVOLTAGE'].apply(lambda x: operatingVoltageDict[x])

dfOHTransformers['Type'] = dfOHTransformers['Type'].astype(str)
dfUGTransformers['Type'] = dfUGTransformers['Type'].astype(str)
dfOHTransformers['Type'] = dfOHTransformers['Type'].apply(lambda x: phasingDict[x])
dfUGTransformers['Type'] = dfUGTransformers['Type'].apply(lambda x: phasingDict[x])

#******************************************************
# Rename col names
dfOHTransformers = dfOHTransformers.rename(columns={'OPERATINGVOLTAGE': 'primary_voltage',
                                                    'Type': 'phasing'})
dfUGTransformers = dfUGTransformers.rename(columns={'OPERATINGVOLTAGE': 'primary_voltage',
                                                    'Type': 'phasing'})
#******************************************************
# Lower case column names
# dfSwitches.columns = map(str.lower, dfSwitches.columns)
dfOHTransformers.columns = map(str.lower, dfOHTransformers.columns)
dfUGTransformers.columns = map(str.lower, dfUGTransformers.columns)

#******************************************************
# REARRANGE COLUMNS
#******************************************************
# *All tables need 'asset_id' - rename index
# OH_TX_COLS = ['asset_id','asset_class_code','id','circuit','install_year','asset_subclass_code','hi',
#               'phasing','primary_voltage','kva','tx_residential','tx_commercial','tx_industrial','device_residential',
#               'device_commercial','device_industrial','upstream_device','prid','in_valley','pcb','banking']
dfOHTransformers = dfOHTransformers[['asset_class_code','id','circuit','install_year','asset_subclass_code','hi','phasing',
                                     'primary_voltage','kva','tx_residential','tx_commercial','tx_industrial',
                                     'device_residential','device_commercial','device_industrial','upstream_device',
                                     'prid','in_valley','pcb','banking','secondary_voltage','fused']]
#'faultindicator','type','units','tx_type_counts','sec_voltage','fused'

# UG_TX_COLS = ['asset_id','asset_subclass_code','asset_class_code','install_year','hi','phasing','prid','circuit',
#               'primary_voltage','kva','in_valley','tx_residential','tx_commercial','tx_industrial','device_residential',
#               'device_commercial','device_industrial','upstream_device','pcb','pedestal','switchable','switch_type','id']
dfUGTransformers = dfUGTransformers[['asset_subclass_code','asset_class_code','install_year','hi','phasing','prid','circuit',
                                     'primary_voltage','kva','in_valley','tx_residential','tx_commercial','tx_industrial',
                                     'device_residential','device_commercial','device_industrial','upstream_device','pcb',
                                     'pedestal','switchable','switch_type','id','secondary_voltage','fused','banking']]


#******************************************************
# OUTPUT DATAFRAMES TO EXCEL FILES
#******************************************************
# OH TX
dfOHTxLater = dfOHTransformers
MasterFile = pd.ExcelWriter(OH_TX_TABLE_TEMPLATE)
dfOHTransformers.to_excel(MasterFile, 'Sheet1')
MasterFile.save()
# UG TX
dfUGTxLater = dfUGTransformers
MasterFile = pd.ExcelWriter(UG_TX_TABLE_TEMPLATE)
dfUGTransformers.to_excel(MasterFile, 'Sheet1')
MasterFile.save()
print('OH and UG Tx, UG Switches analyses completed')

OH and UG Tx, UG Switches analyses completed


In [4]:
#*****************************************************************************************************
# Cell # 3: Poles Analysis - set up template
#*****************************************************************************************************
#**********************************************************************************
#DELETE THIS WHEN COMBINING ALL ASSETS
#**********************************************************************************
# Save future wait times while running
dfPoles = dfPolesV2
#**********************************************************************************
#DELETE THIS WHEN COMBINING ALL ASSETS
#**********************************************************************************
# keep only distribution poles
dfPoles = dfPoles[dfPoles.SUBTYPECD == 1] #1 - Dist(47%), 5-TrafficLights(2%), 7-streetlight(51%)

dropPolesCols = ['SYMBOLROTATION','GPSDATE','SUBTYPECD','LABELTEXT','STRUCTURENUMBER','FEATURE_STATUS',
                 'STREETLIGHT_FACILITY','REPLACED_DATE_MM_DD_YYYY','CONDITION','CONDITION_STATUS','CONDITION_DATE']
dfPoles = drop_columns(dfPoles,dropPolesCols)

# Add additional columns and fill with NaNs
#dfPoles[''] = new_columns(dfPoles, numPolesRows,'')
numPolesRows = len(dfPoles['DEVICENUMBER'])
dfPoles[ASSET_CLASS] = new_columns(dfPoles, numPolesRows,ASSET_CLASS)
dfPoles[ASSET_SUBCLASS] = new_columns(dfPoles, numPolesRows,ASSET_SUBCLASS)
dfPoles['HI'] = new_columns(dfPoles, numPolesRows,'HI')
dfPoles['PHASING'] = new_columns(dfPoles, numPolesRows,'PHASING')
dfPoles['PRID'] = new_columns(dfPoles, numPolesRows,'PRID')
dfPoles['TX'] = new_columns(dfPoles, numPolesRows,'TX')
dfPoles['TX_TYPE'] = new_columns(dfPoles, numPolesRows,'TX_TYPE')
dfPoles['CIRCUIT1'] = new_columns(dfPoles, numPolesRows,'CIRCUIT1')
dfPoles['CIRCUIT2'] = new_columns(dfPoles, numPolesRows,'CIRCUIT2')
dfPoles['CIRCUIT3'] = new_columns(dfPoles, numPolesRows,'CIRCUIT3')
dfPoles['CIRCUIT4'] = new_columns(dfPoles, numPolesRows,'CIRCUIT4')
dfPoles['CIRCUIT5'] = new_columns(dfPoles, numPolesRows,'CIRCUIT5')
dfPoles['CIRCUIT6'] = new_columns(dfPoles, numPolesRows,'CIRCUIT6')
dfPoles['CIRCUIT7'] = new_columns(dfPoles, numPolesRows,'CIRCUIT7')
dfPoles['CIRCUIT8'] = new_columns(dfPoles, numPolesRows,'CIRCUIT8')
dfPoles['IN_VALLEY'] = new_columns(dfPoles, numPolesRows,'IN_VALLEY')
dfPoles['TX_RESIDENTIAL'] = new_columns(dfPoles, numPolesRows,'TX_RESIDENTIAL')
dfPoles['TX_COMMERCIAL'] = new_columns(dfPoles, numPolesRows,'TX_COMMERCIAL')
dfPoles['TX_INDUSTRIAL'] = new_columns(dfPoles, numPolesRows,'TX_INDUSTRIAL')
#dfPoles['HEIGHT'] = new_columns(dfPoles, numPolesRows,'HEIGHT')
dfPoles['NUM_CIRCUITS'] = new_columns(dfPoles, numPolesRows,'NUM_CIRCUITS')
dfPoles['DEVICE'] = new_columns(dfPoles, numPolesRows,'DEVICE')
dfPoles['TX_KVA'] = new_columns(dfPoles, numPolesRows,'TX_KVA')
dfPoles['TX_PHASING'] = new_columns(dfPoles, numPolesRows,'TX_PHASING')

# Fill with values
dfPoles[ASSET_CLASS] = POLES_ASSET_CLASS
dfPoles[ASSET_SUBCLASS] = 'WOOD'

# Poles_class_height table
fileNamePolesClassHeight = 'DomainCodes_PolesClassHeight.xlsx'
# Read Other Device Numbers into dataframes
with pd.ExcelFile(fileNamePolesClassHeight) as xls:
    #dfTopology = pd.read_excel(xlsx, 'Topology', index_col=None, na_values=['NA']) # IGNORE for now
    dfPolesClassHeight = pd.read_excel(xls, 'Sheet1') # 280 rows
#print(dfPoles.shape) 
# Merge tables
dfPoles = dfPoles.merge(dfPolesClassHeight, how='left', on='COMPATIBLEUNITID')

# Rename Pole columns
dfPoles = dfPoles.rename(columns={'DEVICENUMBER':'ID',
                                  'TYPE':'POLE_CLASS',
                                  'INSTALLATIONDATE':'INSTALL_YEAR'})
# Separate year
dfPoles['INSTALL_YEAR'] = dfPoles['INSTALL_YEAR'].apply(lambda x: x.year)

dropPolesCols2 = ['COMPATIBLEUNITID']
dfPoles = drop_columns(dfPoles, dropPolesCols2)
# POLES_COLS = ['asset_id','asset_class_code','asset_subclass_code','install_year','hi character','phasing character',
#               'prid character','pole_class','tx','tx_type','circuit1','circuit2','circuit3','circuit4','in_valley',
#               'tx_residential','tx_commercial','tx_industrial','height','num_circuits','device','tx_kva','id','prid2',
#               'prid3','prid4','tx_pcb']
# ['phasing', 'prid', 'tx', 'tx_type', 'circuit1', 'circuit2', 'circuit3',
#        'circuit4', 'circuit5', 'circuit6', 'circuit7', 'circuit8', 'in_valley',
#        'tx_residential', 'tx_commercial', 'tx_industrial', 'num_circuits',
#        'device', 'tx_kva', 'tx_phasing']

# Lower case column names
dfPoles.columns = map(str.lower, dfPoles.columns)
dfPoles = dfPoles[dfPoles.id.notnull()]
dfPolesLater = dfPoles
#NEED to Rearrange columns - will do when pole attachment info available
# POLES_TABLE = 'IN_POLES.xlsx'
MasterFile = pd.ExcelWriter(POLES_TABLE_TEMPLATE)
dfPoles.to_excel(MasterFile, 'Sheet1')
MasterFile.save()
#print(dfPoles.columns)
print('Poles analysis completed')

Poles analysis completed


In [5]:
#*****************************************************************************************************
# Cell # 4: UG Primary Cable analysis
#*****************************************************************************************************
#**********************************************************************************
#DELETE THIS WHEN COMBINING ALL ASSETS
#**********************************************************************************
# Save future wait times while running
dfCables = dfCablesV2
#**********************************************************************************
#DELETE THIS WHEN COMBINING ALL ASSETS
#**********************************************************************************
# Cables
dropCablesCols = ['ENABLED',  'FEEDERID2', 'FEEDERINFO', 'ELECTRICTRACEWEIGHT', 'LOCATIONID',
                  'LENGTHSOURCE',  'LENGTHUOMCODE', 'LABELTEXT', 'OPERATINGVOLTAGE', 'NOMINALVOLTAGE', 
                  'ISFEEDERTRUNK', 'NEUTRALUSECD', 'FEATURE_STATUS', 'CONDUCTOR_REJUVENATION']
# drop columns not related to Tx
dfCables = drop_columns(dfCables,dropCablesCols)

# Add additional columns and fill with NaNs
numCablesRows = len(dfCables['INSTALLATIONDATE'])

#dfCables
dfCables[ASSET_CLASS] = new_columns(dfCables, numCablesRows, ASSET_CLASS)
dfCables[ASSET_SUBCLASS] = new_columns(dfCables, numCablesRows, ASSET_SUBCLASS)
dfCables['HI'] = new_columns(dfCables,numCablesRows,'HI')
dfCables['PRID'] = new_columns(dfCables, numCablesRows,'PRID')
dfCables['ARRANGEMENT'] = new_columns(dfCables, numCablesRows,'ARRANGEMENT')
dfCables['INSTALLATION'] = new_columns(dfCables, numCablesRows,'INSTALLATION')
dfCables['CONFIG'] = new_columns(dfCables, numCablesRows,'CONFIG')
dfCables['NUM_SPLICES'] = new_columns(dfCables, numCablesRows,'NUM_SPLICES')
dfCables['PRID_RESIDENTIAL'] = new_columns(dfCables, numCablesRows,'PRID_RESIDENTIAL')
dfCables['PRID_COMMERCIAL'] = new_columns(dfCables, numCablesRows,'PRID_COMMERCIAL')
dfCables['PRID_INDUSTRIAL'] = new_columns(dfCables, numCablesRows,'PRID_INDUSTRIAL')
dfCables['WC_CATASTROPHIC_RES'] = new_columns(dfCables, numCablesRows,'WC_CATASTROPHIC_RES')
dfCables['WC_CATASTROPHIC_COMM'] = new_columns(dfCables, numCablesRows,'WC_CATASTROPHIC_COMM')
dfCables['WC_CATASTROPHIC_IND'] = new_columns(dfCables, numCablesRows,'WC_CATASTROPHIC_IND')
dfCables['WC_REPLACEMENT'] = new_columns(dfCables, numCablesRows,'WC_REPLACEMENT')
#dfCables['CABLE_PHASE'] = new_columns(dfCables, numCablesRows,'CABLE_PHASE')

# Rename Cable columns
dfCables = dfCables.rename(columns={'SHAPE_Length':'ID',
                                    'SUBTYPECD':'PHASING',
                                    'INSTALLATIONDATE':'INSTALL_YEAR',
                                    'FEEDERID':'CIRCUIT',
                                    'MEASUREDLENGTH':'LENGTH',
                                    'WIRECOUNT': 'NUM_CABLES',
                                    'PHASEDESIGNATION':'CABLE_PHASE'})
# Separate year
dfCables['INSTALL_YEAR'] = dfCables['INSTALL_YEAR'].apply(lambda x: x.year)
#print(dfCables.head(3))
dfCables[ASSET_SUBCLASS] = 'XLPE'
# Replace Asset class and 'SUBTYPECD' with actual tx types
dictCablesPhasing = {'1':'1','2':'1','3':'1','4':'2','5':'3','6':'Abandon'}
dictCablesPhase = {'0.0':'', '1.0':'B','2.0':'Y','3.0':'YB','4.0':'R','6.0':'RB','7.0':'RYB','':''}

dfCables['PHASING'] = dfCables['PHASING'].astype(str)
dfCables['CABLE_PHASE'] = dfCables['CABLE_PHASE'].astype(str)

#Try using .loc[row_indexer,col_indexer] = value instead
dfCables.loc[:,'PHASING'] = dfCables['PHASING'].apply(lambda x: dictCablesPhasing[x])
dfCables.loc[:,'CABLE_PHASE'] = dfCables['CABLE_PHASE'].apply(lambda x: dictCablesPhase[x])

# Fill in Asset and asset subclass columns
dfCables[ASSET_CLASS] = UG_PRI_CABLE_ASSET_CLASS

# Cables Domain code tables
fileNameDomainCodes_Cables = 'DomainCodes_Cables.xlsx'
# Read Other Device Numbers into dataframes
with pd.ExcelFile(fileNameDomainCodes_Cables) as xls:
    #dfTopology = pd.read_excel(xlsx, 'Topology', index_col=None, na_values=['NA']) # IGNORE for now
    dfCablesDomainCodes = pd.read_excel(xls, 'Sheet1')

dfCables=dfCables.merge(dfCablesDomainCodes, how='left', on='COMPATIBLEUNITID')
#print(dfUGTransformers.head(2))
dropCablesCols2 = ['COMPATIBLEUNITID','Description','Percent']
dfCables = drop_columns(dfCables,dropCablesCols2)

# Lower case column names
dfCables.columns = map(str.lower, dfCables.columns)

# UG_PRI_CABLE_COLS = ['asset_id','id','install_year','hi','asset_subclass_code','asset_class_code','phasing','prid',
#                      'circuit','arrangement','installation','material','cable_size','config','length','num_splices',
#                      'num_cables','prid_residential','prid_commercial','prid_industrial','nominal_voltage',
#                      'wc_prid_catastrophic_res','wc_prid_catastrophic_comm','wc_prid_catastrophic_ind','cable_phase',
#                      'wc_replacement','wc_switching_res','wc_switching_comm','wc_switching_ind','wc_switching_duration']

# print(dfCables.columns)
# ['install_year', 'circuit', 'length', 'num_cables', 'phasing',
#        'cable_phase', 'id', 'asset_class_code', 'asset_subclass_code', 'hi',
#        'prid', 'arrangement', 'installation', 'config', 'num_splices',
#        'prid_residential', 'prid_commercial', 'prid_industrial',
#        'wc_catastrophic_res', 'wc_catastrophic_comm', 'wc_catastrophic_ind',
#        'wc_replacement', 'cable_size', 'material', 'cnshld',
#        'nominal_voltage'],

# UG_PRI_CABLE_TABLE = 'IN_CABLES.xlsx'
# UG_PRI_CABLE_ASSET_CLASS = 'UG_CABLE'

dfCablesLater = dfCables

MasterFile = pd.ExcelWriter(UG_PRI_CABLE_TABLE_TEMPLATE)
dfCables.to_excel(MasterFile, 'Sheet1')
MasterFile.save()
print('Cables analysis completed')

Cables analysis completed


In [6]:
#*****************************************************************************************************
# Cell # 5: Using Machine Learning to map installation years for UG switches/Poles/UG cables
#*****************************************************************************************************
# Lat long df
dfPolesLatLong = dfPolesLatLongV1
dfSwitchesLatLong = dfSwitchesLatLongV1
dfOHCondLatLong = dfOHCondLatLongV1
dfCablesLatLong = dfCablesLatLongV1
dfTxLatLong = dfTxLatLongV1
#**********************************************************************
# UG Switches match
#**********************************************************************
# dfSwitches age need to be fixed
# 20% of UG Switches with install_year 1900
# match with other 80%
#**********************************************************************
dfUGTxLater = dfUGTransformers
# drop XY cols, only 'DEVICENUMB', 'x','y', 'installation_year'
switchXYDropCols = ['FID', 'OBJECTID', 'ANCILLARYR', 'ENABLED', 'WORKORDERI','FIELDVERIF', 'COMMENTS', 
                         'CREATIONUS', 'DATECREATE', 'LASTUSER','DATEMODIFI', 'WORKREQUES', 'DESIGNID', 'WORKLOCATI', 
                         'WMSID','WORKFLOWST', 'WORKFUNCTI',  'FEEDERINFO','ELECTRICTR', 'LOCATIONID', 'GPSDATE', 
                         'GISONUMBER', 'GISOTYPENB','LABELTEXT', 'OWNERSHIP', 'PHASEDESIG','OPERATINGV', 'NOMINALVOL', 
                         'MAXOPERATI', 'MAXCONTINU', 'PRESENTPOS', 'PRESENTP_1', 'PRESENTP_2', 'NORMALPOSI', 'NORMALPO_1',
                         'NORMALPO_2','SCADACONTR', 'SCADAMONIT', 'PREFERREDC', 'TIESWITCHI', 'GANGOPERAT','MANUALLYOP',
                         'FEATURE_ST', 'HYPERLINK', 'HYPERLINK_','SYMBOLROTA', 'INSULATOR_', 'FeederID_1', 'EnergizedP', 
                         'SourceCoun','Loop', 'Tie','COMPATIBLE','SUBTYPECD','FEEDERID', 'FEEDERID2']

#***********
# Tx
#***********
# UG Tx: 2/3/5/7 - 1Ph/Ntwk/Sub/Pad 3Ph [1436,27,4,507: 1642 counts]
dfUGTxXY = pd.DataFrame(dfTxLatLong[(dfTxLatLong.SUBTYPECD == 2) | 
                                    (dfTxLatLong.SUBTYPECD == 3) |
                                    (dfTxLatLong.SUBTYPECD == 5) |
                                    (dfTxLatLong.SUBTYPECD == 7) ])

# OH Tx: 1/9/10 - 1Ph/3Ph/2Ph [1125/510/7: 1347 counts]
dfOHTxXY = pd.DataFrame(dfTxLatLong[(dfTxLatLong.SUBTYPECD == 1) |
                                 (dfTxLatLong.SUBTYPECD == 9) |
                                 (dfTxLatLong.SUBTYPECD == 10)])


txXYDropCols = ['FID', 'OBJECTID', 'ANCILLARYR', 'ENABLED', 'WORKORDERI','FEEDERID','FIELDVERIF', 'COMMENTS', 
                     'CREATIONUS', 'DATECREATE', 'LASTUSER','DATEMODIFI', 'WORKREQUES', 'DESIGNID', 'WORKLOCATI', 'WMSID',
                     'WORKFLOWST', 'WORKFUNCTI', 'FEEDERINFO','ELECTRICTR', 'LOCATIONID', 'SYMBOLROTA', 'GPSDATE', 
                     'GISONUMBER','GISOTYPENB', 'SUBTYPECD', 'LABELTEXT', 'COMPATIBLE', 'OWNERSHIP','PHASEDESIG', 
                     'OPERATINGV', 'NOMINALVOL', 'GROUNDREAC', 'GROUNDRESI','MAGNETIZIN', 'MAGNETIZ_1', 'HIGHSIDEGR', 
                     'HIGHSIDE_1', 'HIGHSIDEPR','LOCATIONTY', 'FAULTINDIC', 'COOLINGTYP', 'FEATURE_ST','KVA', 'UNITS', 
                     'DEMAND_KVA', 'DEMAND_DAT', 'STREET_LIG', 'HIGHSIDECO','LOWSIDECON', 'LOWSIDEGRO', 'LOWSIDEVOL', 
                     'LATITUDE', 'LONGITUDE','RATEDKVA', 'FeederID_1', 'EnergizedP', 'SourceCoun', 'Loop','FEEDERID2']

dfUGTxXY = drop_columns(dfUGTxXY, txXYDropCols)
dfOHTxXY = drop_columns(dfOHTxXY, txXYDropCols)
dfSwitchesXY = drop_columns(dfSwitchesLatLong, switchXYDropCols)

#print(dfSwitchesXY.columns) # 'INSTALLATI','DEVICENUMB', 'x', 'y']
#print(dfTxXY.columns) # 'INSTALLATI','DEVICENUMB', 'x', 'y']
dfUGTxXY = dfUGTxXY.rename(columns={'INSTALLATI':'install_year', 'DEVICENUMB':'id'})
dfSwitchesXY = dfSwitchesXY.rename(columns={'INSTALLATI':'install_year','DEVICENUMB':'id'})

dfSwitchesXY['install_year'] = dfSwitchesXY['install_year'].apply(lambda x: x.year)
dfUGTxXY['install_year'] = dfUGTxXY['install_year'].apply(lambda x: x.year)


#******************************************************
# UG Tx - 1 tx missing installation years - use k-NN
#******************************************************
# separate empty and non-empty 'install_year' values
dfUGTxXY_Train = dfUGTxXY[dfUGTxXY['install_year'].notnull()] 
dfUGTxXY_Empty = dfUGTxXY[dfUGTxXY['install_year'].isnull()] 
print(dfUGTxXY_Empty.shape)
# train the model

dfUGTxXY_Empty.loc[:,'install_year'] = nearest_neighbor(dfUGTxXY_Train, 'x','y','install_year',3,dfUGTxXY_Empty)
dfUGTxXY_new = pd.concat([dfUGTxXY_Train,dfUGTxXY_Empty])
# fill in empty values

# merge with original df
dfUGTxLater = drop_columns(dfUGTxLater,'install_year')
dfUGTxLater = dfUGTxLater.merge(dfUGTxXY_new, how='left', on='id')
dfUGTxLater = drop_columns(dfUGTxLater,['x','y'])
# Machine Learning: empty and filled
#def nearest_neighbor(dfMain, trainX, trainY, classX, neighborCount, dfUnknown):
dfSwitchesXY_Unknown = dfSwitchesXY.loc[dfSwitchesXY['install_year'] == 1900]
dfSwitchesXY_Train = dfSwitchesXY.loc[dfSwitchesXY['install_year'] != 1900]
dfSwitchesXY_Train = pd.concat([dfUGTxXY_new,dfSwitchesXY_Train]) #4093 rows
dfSwitchesXY_Train = dfSwitchesXY_Train[np.isfinite(dfSwitchesXY_Train['install_year'])] # drop NaNs

print('UG Switches Prediction:')
dfSwitchesXY_Unknown.loc[:,'install_year'] = nearest_neighbor(dfSwitchesXY_Train, 'x','y','install_year',1, 
                                                              dfSwitchesXY_Unknown)
# 60% accuracy
dfSwitchesXY_Unknown = drop_columns(dfSwitchesXY_Unknown, ['x','y'])
dfSwitchesXY_Train = drop_columns(dfSwitchesXY_Train, ['x','y'])
dfSwitchesXY = pd.concat([dfSwitchesXY_Unknown,dfSwitchesXY_Train])
print(dfUGSwLater.columns)
# # overwrite template file: Merge with NGN dfSwitches on Device
dfUGSwLater = dfSwitches
print('dfUGSwLater: ', dfUGSwLater.columns)
dfUGSwLater = drop_columns(dfUGSwLater,['install_year'])
dfUGSwLater = dfUGSwLater.merge(dfSwitchesXY, how='left', on='id')
MasterFile = pd.ExcelWriter(UG_SWITCHES_TABLE_TEMPLATE)
dfUGSwLater.to_excel(MasterFile, 'Sheet1')
MasterFile.save()

# Overwrite original UG Tx file
#dfOHTxLater = dfOHTransformers
MasterFile = pd.ExcelWriter(UG_TX_TABLE_TEMPLATE)
dfUGTxLater.to_excel(MasterFile, 'Sheet1')
MasterFile.save()

#**********************************************************************
# UG Cables
#**********************************************************************
# dfCables age need to be fixed
# 60% cables installed in 1900 and 1998
# match with other 80%
#**********************************************************************
cablesXYDropCols = ['FID', 'OBJECTID', 'ENABLED', 'WORKORDERI',  'FIELDVERIF','COMMENTS', 'CREATIONUS', 'DATECREATE', 
                         'LASTUSER', 'DATEMODIFI','WORKREQUES', 'DESIGNID', 'WORKLOCATI', 'WMSID', 'WORKFLOWST','WORKFUNCTI',
                         'FEEDERID2', 'FEEDERINFO', 'ELECTRICTR','LOCATIONID', 'LENGTHSOUR', 'MEASUREDLE', 'LENGTHUOMC', 
                         'WIRECOUNT','GISONUMBER', 'GISOTYPENB', 'LABELTEXT', 'COMPATIBLE','OWNERSHIP', 
                         'PHASEDESIG', 'OPERATINGV', 'NOMINALVOL', 'ISFEEDERTR','NEUTRALUSE', 'FEATURE_ST', 'CONDUCTOR_', 
                         'SHAPE_LEN', 'FeederID_1','EnergizedP', 'SourceCoun', 'Loop']

dfCablesXY = drop_columns(dfCablesLatLong, cablesXYDropCols)
dfCablesXY = dfCablesXY.rename(columns={'xStart': 'x','yStart': 'y','INSTALLATI':'install_year'})

# use UG tx to train and test, fill cable where years <=2002
cableCutoffYr = 2002

#df[~df.C.str.contains("XYZ")] # Remove all 'abandon' phasing
#dfCablesXY = dfCablesXY[dfCablesXY.SUBTYPECD != 6]
dfCablesXY['install_year'] = dfCablesXY['install_year'].apply(lambda x: x.year)

# Machine Learning: empty and filled
#def nearest_neighbor(dfMain, trainX, trainY, classX, neighborCount, dfUnknown):
#print('Shape of dfCablesXY: ', dfCablesXY.shape)# (3888, 9)
dfCablesXY_Unknown = dfCablesXY.loc[dfCablesXY['install_year'] != cableCutoffYr]
dfCablesXY_2002 = dfCablesXY.loc[dfCablesXY['install_year'] == cableCutoffYr]
#print('Shape of dfCablesXY Unknown: ', dfCablesXY_Unknown.shape) #(3853, 9)
dfCablesXY_Train = dfUGTxXY_new
#print('Shape of dfCablesXY_Train: ', dfCablesXY_Train.shape) #(1975, 4)
# print('Before Year NaNs: ', dfCablesXY_Train.shape)
dfCablesXY_Train = dfCablesXY_Train[np.isfinite(dfCablesXY_Train['install_year'])] #one NaN value
# print('After Year NaNs: ', dfCablesXY_Train.shape)

# if combined both UGtx and UGCable > 2002 ~ 45-50% prediction accuracy
# dfCablesXY_Train = dfCablesXY.loc[dfCablesXY['install_year'] == cableCutoffYr]
# dfCablesXY_Train = pd.concat([dfUGTxXY,dfCablesXY_Train]) #4093 rows
# dfCablesXY_Train = dfCablesXY_Train[np.isfinite(dfCablesXY_Train['install_year'])] # drop NaNs
#dfCablesXY_Train = dfCablesXY_Train[np.isfinite(dfCablesXY_Train['FEEDERID'])] # drop NaNs, mostly 'Abandon' phasing

print('UG Cables Prediction:')
dfCablesXY_Unknown.loc[:,'install_year'] = nearest_neighbor(dfCablesXY_Train, 'x','y','install_year',3,dfCablesXY_Unknown)
# 60% accuracy
# cablesXYdropCol = ['xEnd','yEnd','xMid','yMid','SUBTYPECD']
# dfCablesXY_Unknown = drop_columns(dfCablesXY_Unknown, cablesXYdropCol)
# MasterFile = pd.ExcelWriter('V6.1Cables_test.xlsx')
# dfCablesXY_Unknown.to_excel(MasterFile, 'Sheet1')
# MasterFile.save()
# uncomment if UGCable included in training set
# dfCablesXY_Train = drop_columns(dfCablesXY_Train, cablesXYdropCol)

dfCablesXY_all = pd.concat([dfCablesXY_Unknown,dfCablesXY_2002])

#print(dfCablesXY_all.head(4)) # FEEDERID   id  install_year          x   y
dfCablesXY_uniqueX = pd.DataFrame(pd.unique(dfCablesXY_all[['x']].values.ravel()))
#print("all: ",dfCablesXY_all.shape)


dfCablesLater = dfCables
dfCablesLater['x'] = new_columns(dfCablesLater,numCablesRows,'x')
#print('After: ',dfCablesLater.columns)

#print('1:',dfCablesLater.shape) #(3865, 27) | (7869, 50) only if both UGTx and UG cable joined - not recommended
# print(dfCablesXY.shape) # (3888, 8) (3750, 9)
dfCablesLater['x']= dfCablesXY_uniqueX[0].apply(lambda x: x)
#'install_year', 'FEEDERID', 'SUBTYPECD', 'x', 'y', 'xMid', 'yMid', 'xEnd', 'yEnd'
#print('Before: ',dfCablesXY_all.columns)
#dfCablesXY_all = drop_columns(dfCablesXY_all, cablesXYdropCol)
dfCablesXY_all = drop_columns(dfCablesXY_all, ['xMid', 'yMid', 'xEnd', 'yEnd','SUBTYPECD'])
#dfCablesLater = drop_columns(dfCablesLater,cablesXYdropCol)
dfCablesLater = dfCablesLater.merge(dfCablesXY_all, how='left', on='x')
#print(dfCablesXY_all.columns)

dfCablesLater = dfCablesLater.drop_duplicates(['x'], take_last=True)

#used for finding cable installation years with poles in CYME node
dfCablesLater_XY = dfCablesLater
dfCablesLater_XY = drop_columns(dfCablesLater_XY,['install_year_x', 'x','FEEDERID','y'])
dfCablesLater_XY = dfCablesLater_XY.rename(columns = {'install_year_y': 'install_year'})

#Final cleanup
dfCablesLater = drop_columns(dfCablesLater,['install_year_x', 'x','FEEDERID','y'])
dfCablesLater = dfCablesLater.rename(columns = {'install_year_y': 'install_year'})

dfCablesLater = dfCablesLater[np.isfinite(dfCablesLater['install_year'])] #one blank value
#print('After duplicate:',dfCablesLater.shape)
#print(dfCablesXY.columns)
MasterFile = pd.ExcelWriter(UG_PRI_CABLE_TABLE_TEMPLATE)
dfCablesLater.to_excel(MasterFile, 'Sheet1')
MasterFile.save()
print('ML Cables analysis completed')
# print('Before:', dfCablesLatLongV1.shape) # Before: (3888, 50)
# dfCablesLatLongV1 = dfCablesLatLongV1.drop_duplicates(['xStart'], take_last=True)
# print('After:', dfCablesLatLongV1.shape) # After: (3188, 50)


(1, 4)
With k =   1 ,a score of:  45.9665144597
With k =   2 ,a score of:  43.3789954338
With k =   3 ,a score of:  43.9878234399
With k =   5 ,a score of:  42.7701674277
With k =   10 ,a score of:  38.203957382
With k =   20 ,a score of:  31.6590563166
Prediction accuracy:  43.9878234399
UG Switches Prediction:
With k =   1 ,a score of:  58.0448065173
With k =   2 ,a score of:  55.0916496945
With k =   3 ,a score of:  51.0183299389
With k =   5 ,a score of:  46.5376782077
With k =   10 ,a score of:  39.7148676171
With k =   20 ,a score of:  33.7067209776
Prediction accuracy:  58.0448065173
Index(['id', 'asset_subclass_code', 'asset_class_code', 'install_year', 'hi',
       'phasing', 'prid', 'circuit', 'tx_phase', 'in_valley', 'tie_feeder',
       'type'],
      dtype='object')
dfUGSwLater:  Index(['id', 'asset_subclass_code', 'asset_class_code', 'install_year', 'hi',
       'phasing', 'prid', 'circuit', 'tx_phase', 'in_valley', 'tie_feeder',
       'type'],
      dtype='object')
UG C

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:420: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:187: FutureWarning: the take_last=True keyword is deprecated, use keep='last' instead


 20 ,a score of:  27.5494672755
Prediction accuracy:  41.8569254186
ML Cables analysis completed


In [7]:
#*****************************************************************************************************
# Poles Match with installation years - over 54% installed 97-2000
#**********************************************************************
# Match poles with OH Tx
#**********************************************************************
dfPolesLatLong = dfPolesLatLongV1
#dfSwitchesLatLong = dfSwitchesLatLongV1
#dfOHCondLatLong = dfOHCondLatLongV1
#dfCablesLatLong = dfCablesLatLongV1
dfTxLatLong = dfTxLatLongV1

polesXYDropCols = ['FID', 'OBJECTID', 'WORKORDERI','FIELDVERIF', 'COMMENTS','CREATIONUS', 
                        'DATECREATE', 'LASTUSER', 'DATEMODIFI', 'WORKREQUES','DESIGNID', 'WORKLOCATI', 'WMSID', 
                        'WORKFLOWST', 'WORKFUNCTI','SYMBOLROTA', 'GPSDATE', 'GISONUMBER', 'GISOTYPENB', 'SUBTYPECD',
                        'LABELTEXT', 'OWNERSHIP', 'COMPATIBLE', 'STRUCTUREN', 'FEATURE_ST','STREETLIGH', 'REPLACED_D', 
                        'CONDITION', 'CONDITION_','CONDITION1']


# OH Tx: 1/9/10 - 1Ph/3Ph/2Ph [1125/510/7: 1347 counts]
dfPolesLater = dfPoles
dfOHTxLater = dfOHTransformers
dfOHTxXY = pd.DataFrame(dfTxLatLong[(dfTxLatLong.SUBTYPECD == 1) |
                                    (dfTxLatLong.SUBTYPECD == 9) |
                                    (dfTxLatLong.SUBTYPECD == 10)])


txXYDropCols = ['FID', 'OBJECTID', 'ANCILLARYR', 'ENABLED', 'WORKORDERI','FEEDERID','FIELDVERIF', 'COMMENTS', 
                     'CREATIONUS', 'DATECREATE', 'LASTUSER','DATEMODIFI', 'WORKREQUES', 'DESIGNID', 'WORKLOCATI', 'WMSID',
                     'WORKFLOWST', 'WORKFUNCTI', 'FEEDERINFO','ELECTRICTR', 'LOCATIONID', 'SYMBOLROTA', 'GPSDATE', 
                     'GISONUMBER','GISOTYPENB', 'SUBTYPECD', 'LABELTEXT', 'COMPATIBLE', 'OWNERSHIP','PHASEDESIG', 
                     'OPERATINGV', 'NOMINALVOL', 'GROUNDREAC', 'GROUNDRESI','MAGNETIZIN', 'MAGNETIZ_1', 'HIGHSIDEGR', 
                     'HIGHSIDE_1', 'HIGHSIDEPR','LOCATIONTY', 'FAULTINDIC', 'COOLINGTYP', 'FEATURE_ST','KVA', 'UNITS', 
                     'DEMAND_KVA', 'DEMAND_DAT', 'STREET_LIG', 'HIGHSIDECO','LOWSIDECON', 'LOWSIDEGRO', 'LOWSIDEVOL', 
                     'LATITUDE', 'LONGITUDE','RATEDKVA', 'FeederID_1', 'EnergizedP', 'SourceCoun', 'Loop','FEEDERID2']

dfOHTxXY = drop_columns(dfOHTxXY, txXYDropCols)

# drop XY cols, only 'DEVICENUMB', 'x','y', 'installation_year'

dfPolesXY = drop_columns(dfPolesLatLong, polesXYDropCols)
dfPolesXY = dfPolesXY.rename(columns={'DEVICENUMB': 'id','INSTALLATI':'install_year'})
dfOHTxXY = dfOHTxXY.rename(columns={'DEVICENUMB': 'tx','INSTALLATI':'install_year'})
#print(dfPolesXY.columns)
dfPolesXY = dfPolesXY[dfPolesXY.id.notnull()]

dfPolesXY['install_year'] = dfPolesXY['install_year'].apply(lambda x: x.year)
dfOHTxXY['install_year'] = dfOHTxXY['install_year'].apply(lambda x: x.year)
# print(dfPolesXY.head(2))
# print(dfOHTxXY.head(2))

# # Machine Learning: empty and filled
# #def nearest_neighbor(dfMain, trainX, trainY, classX, neighborCount, dfUnknown):
#years_list =[1997,1998,1999,2000]
#df = df[(df.one > 0) | (df.two > 0) | (df.three > 0) & (df.four < 1)]
dfPolesXY_Unknown = dfPolesXY.loc[(dfPolesXY.install_year == 1997) |
                                  (dfPolesXY.install_year == 1998) |
                                  (dfPolesXY.install_year == 1999) |
                                  (dfPolesXY.install_year == 2000) |
                                  (dfPolesXY.install_year.isnull())]

print(dfPolesXY_Unknown.shape) #4771, 4

#******************************************************
# OH Tx - 3 tx missing installation years - use k-NN
#******************************************************
# separate empty and non-empty 'install_year' values
dfOHTxXY_Train = dfOHTxXY[dfOHTxXY['install_year'].notnull()] 
dfOHTxXY_Empty = dfOHTxXY[dfOHTxXY['install_year'].isnull()] 
#print(dfOHTxXY_Empty.shape)
# train the model

dfOHTxXY_Empty.loc[:,'install_year'] = nearest_neighbor(dfOHTxXY_Train, 'x','y','install_year',3,dfOHTxXY_Empty)
dfOHTxXY_new = pd.concat([dfOHTxXY_Train,dfOHTxXY_Empty])
# fill in empty values

# merge with original df
dfOHTxLater = drop_columns(dfOHTxLater,'install_year')
dfOHTxLater = dfOHTxLater.merge(dfOHTxXY_new, how='left', left_on='id', right_on='tx')
dfOHTxLater = drop_columns(dfOHTxLater,['x','y','tx'])

dfPolesXY_Train = dfOHTxXY_new
dfPolesXY_Train = dfPolesXY_Train[np.isfinite(dfPolesXY_Train['install_year'])] # drop NaNs

# print('UG Switches Prediction:')
dfPolesXY_Unknown.loc[:,'install_year'] = nearest_neighbor(dfPolesXY_Train, 'x','y','install_year',3, dfPolesXY_Unknown)
#print(dfPolesXY_Unknown.columns)
dfPolesXY_Unknown = drop_columns(dfPolesXY_Unknown, ['x','y'])
#dfPolesXY_All = dfPolesXY.merge(dfPolesXY_Unknown, how='left', on='x', suffixes=('_all', '_uk'))
dfPolesXY_All = dfPoles.merge(dfPolesXY_Unknown, how='left', on='id', suffixes=('_all', '_uk'))
#dfPolesXY_All = drop_columns(dfPolesXY_All, ['install_year_all'])
#dfPolesXY_All = dfPolesXY_All.rename(columns={'install_year_uk': 'install_year'})
#print(dfPolesXY_All.head(3))
#dfPolesXY_All['install_year_all'] = dfPolesXY_All['install_year_uk'].apply(lambda x: None if math.isnan(x) else x)
#This is a where conditional, saying give me the value for A if A > B, else give me B
# df['A'].where(df['A']>df['B'],df['B'])
dfPolesXY_All['install_year'] = dfPolesXY_All['install_year_uk'].where(dfPolesXY_All['install_year_uk']> 0,
                                                                     dfPolesXY_All['install_year_all'])
dfPolesXY_All = drop_columns(dfPolesXY_All, ['install_year_uk','install_year_all'])
# # 40% accuracy

# Overwrite original OH Tx file
#dfOHTxLater = dfOHTransformers
MasterFile = pd.ExcelWriter(OH_TX_TABLE_TEMPLATE)
dfOHTxLater.to_excel(MasterFile, 'Sheet1')
MasterFile.save()

dfPolesXY_YY = dfPolesXY_All
# Overwrite original pole template file
MasterFile = pd.ExcelWriter(POLES_TABLE_TEMPLATE)
dfPolesXY_All.to_excel(MasterFile, 'Sheet1')
MasterFile.save()
print('Pole installation year matching analysis completed')


(4833, 4)
With k =   1 ,a score of:  37.7981651376
With k =   2 ,a score of:  40.0
With k =   3 ,a score of:  38.3486238532
With k =   5 ,a score of:  36.6972477064
With k =   10 ,a score of:  37.247706422
With k =   20 ,a score of:  34.6788990826
Prediction accuracy:  38.3486238532
With k =   1 ,a score of:  35.1648351648
With k =   2 ,a score of:  36.9963369963
With k =   3 ,a score of:  36.63003663
With k =   5 ,a score of:  38.6446886447
With k =   10 ,a score of:  37.9120879121
With k =   20 ,a score of:  37.1794871795
Prediction accuracy:  36.63003663
Pole installation year matching analysis completed


C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:420: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [8]:
#*****************************************************************************************************
# Cell # 6 : Read all CYME feeders to populate dfAllNodes df - tie Poles with respective feeder ids
#*****************************************************************************************************
# fileName - iterate through entire folder :)
#fileName = '3S3-1_Crestwood_Feeder_Details.xlsx'
#input directory
inputDirectory = 'Metsco_Feeder_Reports'

# define filepath and sort the file list
filesList = glob(os.path.join(inputDirectory, '*.xlsx'))
numFiles = len(filesList)
sortedFileList = sorted(filesList)

# variables
dictFeeders = {}
dfAllNodes_list = pd.DataFrame()

# read text files in tweet_input directory
for f in sortedFileList:

    fileName = os.path.basename(f).split('_')
    FeederKey = fileName[0]
    #print(FeederKey)
    
    if ('$' not in FeederKey):
        # Read CYME Feeder xlsx file into dataframes
        with pd.ExcelFile(f) as xlsx:
            #dfTopology = pd.read_excel(xlsx, 'Topology', index_col=None, na_values=['NA']) # IGNORE for now
            dfTopology = pd.read_excel(xlsx, 'Topology') # 280 rows
            dfSpotLoads = pd.read_excel(xlsx, 'Spot Loads') # Tot:239 - R/Y/B: 116/108/103 values; based on phases
            dfLoads = pd.read_excel(xlsx, 'Loads') # 239 rows; 'Spot Number\n' col contains unique tx ids
            dfCables = pd.read_excel(xlsx, 'Cables')
            dfSwitches = pd.read_excel(xlsx, 'Switches') # 41 items
            dfNodes = pd.read_excel(xlsx, 'Nodes') # 249 items
            dfOHlines = pd.read_excel(xlsx, 'OverheadLinesByPhase') #Neutral - 94, Section Id - 381
            dfFuses = pd.read_excel(xlsx, 'Fuses') # 44 items

            # # Strip '\n' from column headers
            dfTopology.rename(columns=lambda x: x.replace('\n',''), inplace=True)
            dfSpotLoads.rename(columns=lambda x: x.replace('\n',''), inplace=True)
            dfLoads.rename(columns=lambda x: x.replace('\n',''), inplace=True)
            dfCables.rename(columns=lambda x: x.replace('\n',''), inplace=True)
            dfSwitches.rename(columns=lambda x: x.replace('\n',''), inplace=True)
            dfNodes.rename(columns=lambda x: x.replace('\n',''), inplace=True)
            dfOHlines.rename(columns=lambda x: x.replace('\n',''), inplace=True)
            dfFuses.rename(columns=lambda x: x.replace('\n',''), inplace=True)
            #print(dfNodes.columns)
            dfAllNodes_list = dfAllNodes_list.append(dfNodes)

print('Number of Feeders: ', numFiles)

Number of Feeders:  36


In [9]:
#*****************************************************************************************************
# Cell # 7: Finding circuits: For both poles and UG Cables by matching with all nodes 
#*****************************************************************************************************
from collections import defaultdict
# Lat long df
dfPolesLatLong = dfPolesLatLongV1
# dfSwitchesLatLong = dfSwitchesLatLongV1
# dfCablesLatLong = dfCablesLatLongV1
# dfTxLatLong = dfTxLatLongV1

#print(dfAllNodes_list.columns)
dfPolesXY_All = dfPolesXY_YY
dfAllNodes_list = dfAllNodes_list.rename(columns={'Network Id': 'CIRCUIT',
                                                  'Node Id': 'xy'})

dfAllNodes_list['NodeID_x'], dfAllNodes_list['NodeID_y'] = zip(*dfAllNodes_list['xy'].
                                                               apply(lambda x: x.split('_') if '_' in x else (x, np.nan)))

# Convert circuit ids, fuse ids, switch ids and/or drop them
#dfAllNodes_list['NodeID_x'] = dfAllNodes_list['NodeID_x'].astype(float)
dfAllNodes_list.loc[:,'NodeID_x'] = dfAllNodes_list.loc[:,'NodeID_x'].convert_objects(convert_numeric=True)
dfAllNodes_list.loc[:,'NodeID_y'] = dfAllNodes_list.loc[:,'NodeID_y'].convert_objects(convert_numeric=True)

dfAllNodes_list = dfAllNodes_list[dfAllNodes_list.NodeID_x.notnull()]
#print(dfAllNodes_XY_remain.isnull().sum())
dfAllNodes_list = dfAllNodes_list[dfAllNodes_list.NodeID_y.notnull()]

dfPoles.columns = map(str.lower, dfPoles.columns)

polesLatLongDropCols = ['FID', 'OBJECTID', 'WORKORDERI', 'INSTALLATI', 'FIELDVERIF', 'COMMENTS','CREATIONUS', 
                        'DATECREATE', 'LASTUSER', 'DATEMODIFI', 'WORKREQUES','DESIGNID', 'WORKLOCATI', 'WMSID', 
                        'WORKFLOWST', 'WORKFUNCTI','SYMBOLROTA', 'GPSDATE', 'GISONUMBER', 'GISOTYPENB', 'SUBTYPECD',
                        'LABELTEXT', 'OWNERSHIP', 'COMPATIBLE', 'STRUCTUREN', 'FEATURE_ST','STREETLIGH', 'REPLACED_D', 
                        'CONDITION', 'CONDITION_','CONDITION1']

# 'DEVICENUMB', 'x', 'y'
dfPolesLatLong = drop_columns(dfPolesLatLong, polesLatLongDropCols)

# ['DEVICENUMB', 'x', 'y', 'xy']
dfPolesLatLong = dfPolesLatLong.rename(columns={'DEVICENUMB': 'POLE_ID'})
dropMoreLatLongCols =['x','y']
#dropMoreWireLatLongCols =['x','y','xMid','yMid','xEnd', 'yEnd']
dropMoreWireLatLongCols =['xStart','yStart','xMid','yMid','xEnd', 'yEnd']

dfPoleIDs = dfPolesLatLong['POLE_ID']
dictPoleIDs = defaultdict(list)

xDelta = 5
yDelta = 5

for poleID, Px, Py in zip(dfPolesLatLong['POLE_ID'],dfPolesLatLong['x'],dfPolesLatLong['y']):
    for circuitID, Nx, Ny in zip(dfAllNodes_list['CIRCUIT'],dfAllNodes_list['NodeID_x'],dfAllNodes_list['NodeID_y']):
        if(abs(Px-Nx) < xDelta and abs(Py-Ny) < yDelta):
            if circuitID not in dictPoleIDs[poleID]:
                dictPoleIDs[poleID].append(circuitID)

dfPolesNodes = pd.DataFrame.from_dict(dictPoleIDs, orient='index')

#df.reset_index(level=0, inplace=True)
dfPolesNodes.reset_index(level=0, inplace=True)
dfPolesNodes = dfPolesNodes.rename(columns={0:'CIRCUIT1', 
                                            1:'CIRCUIT2',
                                            2:'CIRCUIT3',
                                            'index':'POLE_ID'})
#print(dfPolesNodes.columns) #['POLE_ID', 'CIRCUIT1', 'CIRCUIT2', 'CIRCUIT3']

MasterFile = pd.ExcelWriter('V8_PolesNodes.xlsx')
dfPolesNodes.to_excel(MasterFile, 'Sheet1')
MasterFile.save()
#print('Pole Nodes')

dfPolesXY_All = dfPolesXY_All.merge(dfPolesNodes, how='left', left_on='id', right_on='POLE_ID')
#print(dfPolesXY_All.columns)

MasterFile = pd.ExcelWriter(POLES_TABLE_TEMPLATE)
dfPolesXY_All.to_excel(MasterFile, 'Sheet1')
MasterFile.save()
print('Pole installation year matching analysis completed')


Pole installation year matching analysis completed


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:21: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:22: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [10]:
#********************************************
# Feeder loading table
#********************************************

RES_LOAD = 'feeder_residential_load'
COM_LOAD = 'feeder_commercial_load'
IND_LOAD = 'feeder_industrial_load'

fdr_dropCols = ['asset_class_code','asset_subclass_code', 'hi', 'primary_voltage', 'tx_residential', 
                'tx_commercial','tx_industrial', 'device_residential', 'device_commercial','device_industrial', 
                'upstream_device','prid', 'in_valley', 'pcb', 'install_year','id']

dfFdrs_total = 0
dfFdr_OHtx = dfOHTxLater
dfFdr_UGtx = dfUGTxLater
dfFdr_OHtx = drop_columns(dfFdr_OHtx,fdr_dropCols)
dfFdr_UGtx = drop_columns(dfFdr_UGtx,fdr_dropCols)
dfFdr_OHtx = drop_columns(dfFdr_OHtx,['banking'])
dfFdr_UGtx = drop_columns(dfFdr_UGtx,['pedestal','switchable', 'switch_type'])

#This is a where conditional, saying give me the value for A if A > B, else give me B
# df['A'].where(df['A']>df['B'],df['B'])
#http://stackoverflow.com/questions/27041724/using-conditional-to-generate-new-column-in-pandas-dataframe
#df.loc[(df['used'] >0.0) & (df['used'] < 1.0), 'alert'] = 'Partial'
#print(dfFdr_OHtx_Rx.aggregate(np.sum))
#UGTx: 1-2Ph <=100kVA #OHTx: 1Ph/2Ph < 150kVA
#df.groupby(['col5','col2']).size().reset_index()
dfFdr_OHtx_Rx = dfFdr_OHtx[(dfFdr_OHtx['phasing'] != 3) & (dfFdr_OHtx['kva'] <=150)]
dfFdr_UGtx_Rx = dfFdr_UGtx[(dfFdr_UGtx['phasing'] != 3) & (dfFdr_UGtx['kva'] <=100)]
dfFdr_Rx = pd.concat([dfFdr_OHtx_Rx,dfFdr_UGtx_Rx])
dfFdr_Rx = dfFdr_Rx.rename(columns={'kva': RES_LOAD})
# UGtx: 3Ph: 100-350kVA, 1Ph: <100kVA; group it by less than 350kVA # OHtx: 3Ph, 
dfFdr_OHtx_Med = dfFdr_OHtx[(dfFdr_OHtx['phasing'] == 3)]
dfFdr_UGtx_Med = dfFdr_UGtx[(dfFdr_UGtx['phasing'] == 3) & (dfFdr_UGtx['kva'] <=100) |
                            (dfFdr_UGtx['phasing'] != 3) & ((dfFdr_UGtx['kva'] > 100) & (dfFdr_UGtx['kva'] <=350))]
dfFdr_Med = pd.concat([dfFdr_OHtx_Med,dfFdr_UGtx_Med])
dfFdr_Med = dfFdr_Med.rename(columns={'kva': COM_LOAD})
#UGTx: Greater than 350kVA
dfFdr_UGtx_Large = dfFdr_UGtx[(dfFdr_UGtx['kva'] > 350)]
dfFdr_Large = dfFdr_UGtx_Large
dfFdr_Large = dfFdr_Large.rename(columns={'kva': IND_LOAD})
#
dfFdrs_Loads = pd.concat([dfFdr_Rx, dfFdr_Med, dfFdr_Large])
dfFdrs_Loads = drop_columns(dfFdrs_Loads, ['phasing'])
dfFdrs_total = pd.DataFrame(dfFdrs_Loads.groupby('circuit').sum()).reset_index()

#50% nameplate rating
dfFdrs_total[RES_LOAD] = dfFdrs_total[RES_LOAD].apply(lambda x: x/2)
dfFdrs_total[COM_LOAD] = dfFdrs_total[COM_LOAD].apply(lambda x: x/2)
dfFdrs_total[IND_LOAD] = dfFdrs_total[IND_LOAD].apply(lambda x: x/2)

# asset_id, feeder,scada_switch_count,manual_switch_count,da_switch_count,
#feeder_residential_load,feeder_commercial_load,feeder_industrial_load,configuration,
#switching_sections 
#Fdrs_key = pd.Series(dfFdr_Load['circuit'].values.ravel()).unique()

dfFdrs_total['circuit'] = dfFdrs_total['circuit'].apply(lambda x: re.sub('[\s+]', '', x))
# print(dfFdrs_total.head(3))
# print(dfFdrs_total.columns)

# Switches
dfSwitchesCount = dfSwitchesV2

dropSwitchesColsCount = ['ANCILLARYROLE','ENABLED','FEEDERINFO','ELECTRICTRACEWEIGHT','LOCATIONID','GPSDATE','LABELTEXT',
                         'OPERATINGVOLTAGE', 'NOMINALVOLTAGE', 'MAXOPERATINGVOLTAGE','MAXCONTINUOUSCURRENT','PRESENTPOSITION_R', 
                         'PRESENTPOSITION_Y', 'PRESENTPOSITION_B','NORMALPOSITION_R','NORMALPOSITION_Y','NORMALPOSITION_B', 
                         'SCADACONTROLID', 'SCADAMONITORID','PREFERREDCIRCUITSOURCE','TIESWITCHINDICATOR','GANGOPERATED', 
                         'MANUALLYOPERATED','FEATURE_STATUS','HYPERLINK','HYPERLINK_PGDB','SYMBOLROTATION','INSULATOR_MATERIAL',
                         'INSTALLATIONDATE','FEEDERID2','SUBTYPECD','COMPATIBLEUNITID','PHASEDESIGNATION']

dfSwitchesCount = drop_columns(dfSwitchesCount, dropSwitchesColsCount)
dfSwitchesCircuit = pd.DataFrame(dfSwitchesCount.groupby('FEEDERID').count()).reset_index()

dfSwitchesCircuit = dfSwitchesCircuit.rename(columns={'FEEDERID': 'circuit', 'DEVICENUMBER': 'manual_switch_count'})
dfFdrs_total = dfFdrs_total.merge(dfSwitchesCircuit, how='left', on='circuit')
dfFdrs_total = dfFdrs_total.rename(columns={'circuit':'feeder'})
#print(dfFdrs_total.head(4))
# Remaining columns 
dfFdrs_total['scada_switch_count'] = 0
dfFdrs_total['da_switch_count'] = 0
dfFdrs_total['configuration'] = 'None'
dfFdrs_total['switching_sections'] = 0
dfFdrs_total['asset_id'] = pd.Series(range(0,len(dfFdrs_total['feeder'])))

dfFdrs_total = dfFdrs_total[['asset_id','feeder','scada_switch_count','manual_switch_count','da_switch_count',
                             'feeder_residential_load','feeder_commercial_load','feeder_industrial_load',
                             'configuration','switching_sections']]
dfFdrs_total=dfFdrs_total.fillna(0)
MasterFile = pd.ExcelWriter('V8_FeederLoads.xlsx')
dfFdrs_total.to_excel(MasterFile, 'Sheet1')
MasterFile.save()
print('Feeder load table completed')


Feeder load table completed


In [24]:
print('begins')
RES_LOAD = 'feeder_residential_load'
MED_COM_LOAD = 'feeder_small_med_commercial_load'
LARGE_LOAD = 'feeder_large_commercial_load'

with pd.ExcelFile('V5_PRID_TX_LOOKUP.xlsx') as xls:
    dfPRIDtx = pd.read_excel(xls, 'Sheet1')

#remove the column and rename it in near future
dfPRIDtx = dfPRIDtx.rename(columns={'transformerid':'id'})
#dfPRIDtx = drop_columns(dfPRIDtx, ['TransformerID_x', 'TransformerID_y'])

dfPRID_OHtx = dfOHTxLater
dfPRID_UGtx = dfUGTxLater
dfPRID_OHtx = drop_columns(dfPRID_OHtx,['prid'])
dfPRID_UGtx = drop_columns(dfPRID_UGtx,['prid'])

dfPRID_OHtx = dfPRID_OHtx.merge(dfPRIDtx, how='left', on='id')
dfPRID_UGtx = dfPRID_UGtx.merge(dfPRIDtx, how='left', on='id')

#PRIDdropCols = ['tx_residential','tx_commercial','tx_industrial']
PRIDdropCols = [RES_LOAD,MED_COM_LOAD,LARGE_LOAD]

dfPRID_OHtx = drop_columns(dfPRID_OHtx, PRIDdropCols)

# dfPRID_OHtx = dfPRID_OHtx.rename(columns={RES_LOAD:'tx_residential',
#                                           MED_COM_LOAD:'tx_med_com_load',
#                                           LARGE_LOAD:'tx_large_commercial_load',
#                                           'PRID':'prid'})

#'secondary_voltage','fused','banking'


dfPRID_OHtx['kva'] = dfPRID_OHtx['kva'].astype(float)
dfPRID_UGtx['kva'] = dfPRID_UGtx['kva'].astype(float)
dfPRID_OHtx['phasing'] = dfPRID_OHtx['phasing'].astype(float)
dfPRID_UGtx['phasing'] = dfPRID_UGtx['phasing'].astype(float)

def phasing_kva(phasing,kva,custClass):
    if((kva > 350) & (custClass=='classLarge')):
        return kva/2
    elif((phasing == 3) & (kva <=350) & (custClass=='classMed')):
        return kva/2
    elif((phasing != 3) & (kva <=150) & (custClass=='classRx')):
        return kva/2
    else:
        return 0
# def valuation_formula(x, y):
#     return x * y * 0.5
#df['price'] = df.apply(lambda row: valuation_formula(row['x'], row['y']), axis=1)

dfPRID_OHtx['tx_residential'] = dfPRID_OHtx.apply(lambda row: phasing_kva( row['phasing'],row['kva'], 'classRx'), axis=1)
dfPRID_OHtx['tx_commercial'] = dfPRID_OHtx.apply(lambda row: phasing_kva( row['phasing'],row['kva'], 'classMed'), axis=1)
dfPRID_OHtx['tx_industrial'] = dfPRID_OHtx.apply(lambda row: phasing_kva( row['phasing'],row['kva'], 'classLarge'), axis=1)

dfPRID_UGtx['tx_residential'] = dfPRID_UGtx.apply(lambda row: phasing_kva( row['phasing'],row['kva'], 'classRx'), axis=1)
dfPRID_UGtx['tx_commercial'] = dfPRID_UGtx.apply(lambda row: phasing_kva( row['phasing'],row['kva'], 'classMed'), axis=1)
dfPRID_UGtx['tx_industrial'] = dfPRID_UGtx.apply(lambda row: phasing_kva( row['phasing'],row['kva'], 'classLarge'), axis=1)

# for index, row in dfPRID_OHtx.iterrows():
#     row['tx_residential'] = phasing_kva(row['phasing'],row['kva'])

#device_residential-device_commercial-device_industrial	upstream_device	prid
dfPRID_OHtx['in_valley'] = 'No'
dfPRID_OHtx['pcb'] ='refer Eval data'
dfPRID_OHtx['hi'] = 'NA'
dfPRID_OHtx.index.name = 'asset_id'
dfPRID_OHtx = dfPRID_OHtx[['asset_class_code','id','circuit','install_year','asset_subclass_code','hi','phasing',
                           'primary_voltage','kva','tx_residential','tx_commercial','tx_industrial','device_residential',
                           'device_commercial','device_industrial','upstream_device','prid','in_valley','pcb','banking',
                           'secondary_voltage','fused']]

MasterFile = pd.ExcelWriter(OH_TX_TABLE_TEMPLATE)
dfPRID_OHtx.to_excel(MasterFile, 'Sheet1')
MasterFile.save()

dfPRID_UGtx = drop_columns(dfPRID_UGtx, PRIDdropCols)

#device_residential-device_commercial-device_industrial	upstream_device	prid
dfPRID_UGtx['pedestal'] = 'NA'
dfPRID_UGtx['pcb'] ='refer Eval data'
dfPRID_UGtx['switch_type'] = 'NA'
dfPRID_UGtx['hi'] = 'NA'
dfPRID_UGtx['switch_type'] = 'NA'
dfPRID_UGtx['switchable'] = 'NA'
dfPRID_UGtx.index.name = 'asset_id'

dfPRID_UGtx = dfPRID_UGtx[['asset_subclass_code','asset_class_code','install_year','hi','phasing','prid','circuit',
                           'primary_voltage','kva','tx_residential','tx_commercial','tx_industrial','device_residential',
                           'device_commercial','device_industrial','upstream_device','pcb','pedestal','switchable',
                           'switch_type','id','secondary_voltage','fused','banking']]

dfPRID_UGtx = dfPRID_UGtx[dfPRID_UGtx['asset_subclass_code'] !='NETWORK_SUBMERSIBLE']

MasterFile = pd.ExcelWriter(UG_TX_TABLE_TEMPLATE)
dfPRID_UGtx.to_excel(MasterFile, 'Sheet1')
MasterFile.save()
print('UG and OH Tx PRIDs matched and loading types classified')

begins
UG and OH Tx PRIDs matched and loading types classified


In [48]:
#*****************************************************************************************************
# Cell # 7.1: Pole attachments: Poles XY with TX, Switches
#*****************************************************************************************************
# Switches
dfPolesLatLong = dfPolesLatLongV1
dfSwitchesLatLong = dfSwitchesLatLongV1
dfTxLatLong = dfTxLatLongV1

dfPolesXY_aa = dfPolesXY_All

#print(dfPolesXY_aa.columns)
#print(dfPolesXY_aa.columns)
polesLatLongDropCols = ['FID', 'OBJECTID', 'WORKORDERI', 'INSTALLATI', 'FIELDVERIF', 'COMMENTS','CREATIONUS', 
                        'DATECREATE', 'LASTUSER', 'DATEMODIFI', 'WORKREQUES','DESIGNID', 'WORKLOCATI', 'WMSID', 
                        'WORKFLOWST', 'WORKFUNCTI','SYMBOLROTA', 'GPSDATE', 'GISONUMBER', 'GISOTYPENB', 'SUBTYPECD',
                        'LABELTEXT', 'OWNERSHIP', 'COMPATIBLE', 'STRUCTUREN', 'FEATURE_ST','STREETLIGH', 'REPLACED_D', 
                        'CONDITION', 'CONDITION_','CONDITION1']

# 'DEVICENUMB', 'x', 'y'
dfPolesLatLong = drop_columns(dfPolesLatLong, polesLatLongDropCols)
dfPolesXY_aa = drop_columns(dfPolesXY_aa, ['device','circuit1','circuit2','circuit3','tx','tx_kva','tx_phasing'])
#Change datatype to str for concatenation
dfPolesLatLong['x'] = dfPolesLatLong['x'].astype(str)
dfPolesLatLong['y'] = dfPolesLatLong['y'].astype(str)
dfPolesLatLong['xy'] = dfPolesLatLong['x']+'-'+dfPolesLatLong['y']
# ['DEVICENUMB', 'x', 'y', 'xy']
dfPolesLatLong = dfPolesLatLong.rename(columns={'DEVICENUMB': 'POLE_ID'})
dropMoreLatLongCols =['x','y']
#dropMoreWireLatLongCols =['x','y','xMid','yMid','xEnd', 'yEnd']
dfPolesLatLong = drop_columns(dfPolesLatLong, dropMoreLatLongCols)

#SWITCHES
switchLatLongDropCols = ['FID', 'OBJECTID', 'ANCILLARYR', 'ENABLED', 'WORKORDERI', 'INSTALLATI','FIELDVERIF', 'COMMENTS', 
                         'CREATIONUS', 'DATECREATE', 'LASTUSER','DATEMODIFI', 'WORKREQUES', 'DESIGNID', 'WORKLOCATI', 
                         'WMSID','WORKFLOWST', 'WORKFUNCTI',  'FEEDERINFO','ELECTRICTR', 'LOCATIONID', 'GPSDATE', 
                         'GISONUMBER', 'GISOTYPENB','LABELTEXT', 'OWNERSHIP', 'PHASEDESIG','OPERATINGV', 'NOMINALVOL', 
                         'MAXOPERATI', 'MAXCONTINU', 'PRESENTPOS', 'PRESENTP_1', 'PRESENTP_2', 'NORMALPOSI', 'NORMALPO_1',
                         'NORMALPO_2','SCADACONTR', 'SCADAMONIT', 'PREFERREDC', 'TIESWITCHI', 'GANGOPERAT','MANUALLYOP',
                         'FEATURE_ST', 'HYPERLINK', 'HYPERLINK_','SYMBOLROTA', 'INSULATOR_', 'FeederID_1', 'EnergizedP', 
                         'SourceCoun','Loop', 'Tie','COMPATIBLE','SUBTYPECD']

dfSwitchesLatLong = drop_columns(dfSwitchesLatLong, switchLatLongDropCols)
dfSwitchesLatLong = dfSwitchesLatLong.rename(columns={'DEVICENUMB': 'device',
                                                      'FEEDERID': 'CIRCUIT1',
                                                      'FEEDERID2': 'CIRCUIT2'})
#Change datatype to str for concatenation
dfSwitchesLatLong['x'] = dfSwitchesLatLong['x'].astype(str)
dfSwitchesLatLong['y'] = dfSwitchesLatLong['y'].astype(str)
dfSwitchesLatLong['xy'] = dfSwitchesLatLong['x']+'-'+dfSwitchesLatLong['y']
# print(dfSwitchesLatLong.columns)
#['FEEDERID', 'FEEDERID2', 'SUBTYPECD','DEVICENUMB', 'x','y', 'xy']
dfSwitchesLatLong = drop_columns(dfSwitchesLatLong, dropMoreLatLongCols)

# Poles and Switch match - 98 matches
dfPolesSwitchLatLong = dfPolesLatLong.merge(dfSwitchesLatLong, how='left', on='xy')
dfPolesSwitchLatLong = dfPolesSwitchLatLong[dfPolesSwitchLatLong.device.notnull()]
# Pole-Switch Output
MasterFile = pd.ExcelWriter('V7_PolesSwitches.xlsx')
dfPolesSwitchLatLong.to_excel(MasterFile, 'Sheet1')
MasterFile.save()

# drop Pole switch columns
dfPolesSwitchLatLong = drop_columns(dfPolesSwitchLatLong, ['xy','CIRCUIT1', 'CIRCUIT2'])
dfPolesXY_aa = dfPolesXY_aa.merge(dfPolesSwitchLatLong, how='left', left_on='id', right_on='POLE_ID')

#print('dfPolesXY_aa cols:',dfPolesXY_aa.columns)
#**********************************************************************
# Tx
#**********************************************************************
txLatLongDropCols = ['FID', 'OBJECTID', 'ANCILLARYR', 'ENABLED', 'WORKORDERI', 'INSTALLATI','FIELDVERIF', 'COMMENTS', 
                     'CREATIONUS', 'DATECREATE', 'LASTUSER','DATEMODIFI', 'WORKREQUES', 'DESIGNID', 'WORKLOCATI', 'WMSID',
                     'WORKFLOWST', 'WORKFUNCTI', 'FEEDERINFO','ELECTRICTR', 'LOCATIONID', 'SYMBOLROTA', 'GPSDATE', 
                     'GISONUMBER','GISOTYPENB', 'SUBTYPECD', 'LABELTEXT', 'COMPATIBLE', 'OWNERSHIP','PHASEDESIG', 
                     'OPERATINGV', 'NOMINALVOL', 'GROUNDREAC', 'GROUNDRESI','MAGNETIZIN', 'MAGNETIZ_1', 'HIGHSIDEGR', 
                     'HIGHSIDE_1', 'HIGHSIDEPR','LOCATIONTY', 'FAULTINDIC', 'COOLINGTYP', 'FEATURE_ST','KVA', 'UNITS', 
                     'DEMAND_KVA', 'DEMAND_DAT', 'STREET_LIG', 'HIGHSIDECO','LOWSIDECON', 'LOWSIDEGRO', 'LOWSIDEVOL', 
                     'LATITUDE', 'LONGITUDE','RATEDKVA', 'FeederID_1', 'EnergizedP', 'SourceCoun', 'Loop','FEEDERID2']

dfTxLatLong = drop_columns(dfTxLatLong, txLatLongDropCols)

dfTxLatLong = dfTxLatLong.merge(dfOHTransformers, how='left', left_on='DEVICENUMB', right_on='id')

txLatLongDropCols2 = ['DEVICENUMB','asset_class_code','asset_subclass_code', 'hi', 'primary_voltage','device_residential', 
                      'device_commercial','device_industrial','upstream_device', 'prid', 'in_valley', 'pcb','banking',
                      'FEEDERID', 'install_year','tx_residential','tx_commercial', 'tx_industrial']

dfTxLatLong = drop_columns(dfTxLatLong, txLatLongDropCols2)

#dfPRID_UGtx = dfPRID_UGtx.merge(dfPRIDtx, how='left', on='id')
dfTxLatLong = dfTxLatLong.merge(dfPRIDtx, how='left', on='id')
dfTxLatLong = drop_columns(dfTxLatLong, ['fused', 'secondary_voltage'])
dfTxLatLong = dfTxLatLong.rename(columns={'prid':'PRID'})
#print(dfTxLatLong.isnull().sum())
dfTxLatLong = dfTxLatLong.rename(columns={'feeder_residential_load':'tx_residential', 
                                          'feeder_small_med_commercial_load':'tx_commercial',
                                          'feeder_large_commercial_load':'tx_industrial'})

#print('dfTxLatLong',dfTxLatLong.columns)
# print(dfTxLatLong.head(3))
#Change datatype to str for concatenation
dfTxLatLong['x'] = dfTxLatLong['x'].astype(str)
dfTxLatLong['y'] = dfTxLatLong['y'].astype(str)
dfTxLatLong['xy'] = dfTxLatLong['x']+'-'+dfTxLatLong['y']

#['FEEDERID','DEVICENUMB', 'x','y', 'xy']
dfTxLatLong = drop_columns(dfTxLatLong, dropMoreLatLongCols)
dfTxLatLong = dfTxLatLong.rename(columns={'circuit': 'CIRCUIT1', 'id':'txID'})
dfPolesTxLatLong = drop_columns(dfPolesTxLatLong,['tx_residential','tx_commercial', 'tx_industrial'])
dfPolesTxLatLong = dfPolesLatLong.merge(dfTxLatLong, how='left', on='xy') # Poles and Tx match - 1553 matches out of 1642 [89 no matches ~ 5%]
dfPolesTxLatLong = dfPolesTxLatLong[dfPolesTxLatLong.txID.notnull()]
#print('dfPolesTxLatLong',dfPolesTxLatLong.columns)
# will merge each df independently
dfPolesTxLatLong = drop_columns(dfPolesTxLatLong, ['xy','CIRCUIT1'])

dfPolesXY_aa = drop_columns(dfPolesXY_aa, ['tx_residential', 'tx_commercial', 'tx_industrial'])

dfPolesXY_aa = dfPolesXY_aa.merge(dfPolesTxLatLong, how='left', left_on='id', right_on='POLE_ID')
#print('dfPolesXY_aa with TX cols:',dfPolesXY_aa.columns)
dfPolesXY_aa = dfPolesXY_aa.rename(columns={'phasing_x': 'phasing', 
                                            'phasing_y':'tx_phasing',
                                            'txID':'tx',
                                            'kva':'tx_kva'})
#                                             'CIRCUIT1_x':'circuit1',
#                                             'CIRCUIT2_x':'circuit2',
#                                             'CIRCUIT3_x':'circuit3'})

# dfNPTS = pd.concat([dfPolesNodes, dfPolesTxSwitchLatLong])
# dfNPTS = drop_columns(dfNPTS, ['xy','CIRCUIT1','CIRCUIT2','CIRCUIT3'])
# dfPolesXY_aa = dfPolesXY_aa.merge(dfNPTS, how='left', on='POLE_ID')

dfPolesXY_aa['phasing'] = dfPolesXY_aa['tx_phasing']
#dfPolesXY_aa['num_circuits'] ='refer Eval data'
dfPolesXY_aa['in_valley'] = 'NA'
dfPolesXY_aa['hi'] = 'NA'

polesXY_dropCols = ['POLE_ID_x','POLE_ID_y','POLE_ID','prid']
dfPolesXY_aa = drop_columns(dfPolesXY_aa, polesXY_dropCols)
dfPolesXY_aa.columns = map(str.lower, dfPolesXY_aa.columns)

def isCircuit(c1, c2, c3):
    if c3:
        return 3
    elif(c2):
        return 2
    else:
        return 1
    
dfPolesXY_aa['num_circuits'] = dfPolesXY_aa.apply(lambda row: 
                                                  isCircuit(row['circuit1'],row['circuit2'],row['circuit3']), axis=1)
#print(dfPolesXY_aa.columns)

#rearrange
dfPolesXY_aa = dfPolesXY_aa[['install_year','id','asset_class_code','asset_subclass_code','hi','phasing','prid','tx',
                             'tx_type','circuit1','circuit2','circuit3','circuit4','circuit5','circuit6','circuit7',
                             'circuit8','in_valley','tx_residential','tx_commercial','tx_industrial','num_circuits',
                             'device','tx_kva','tx_phasing','height','pole_class']]

MasterFile = pd.ExcelWriter(POLES_TABLE_TEMPLATE)
dfPolesXY_aa.to_excel(MasterFile, 'Sheet1')
MasterFile.save()
print('Pole attachments analysis completed')

#Transformer-Poles match output
# MasterFile = pd.ExcelWriter('V5_PolesTx.xlsx')
# dfPolesTxLatLong.to_excel(MasterFile, 'Sheet1')
# MasterFile.save()

#Transformer-Switch-Poles match output
# dfPolesTxSwitchLatLong.columns = map(str.lower, dfPolesTxSwitchLatLong.columns)
# MasterFile = pd.ExcelWriter('V7_PolesTxSwitch.xlsx')
# dfPolesTxSwitchLatLong.to_excel(MasterFile, 'Sheet1')
# MasterFile.save()

#NPTS(NodesPolesTxSwitch) match output
# MasterFile = pd.ExcelWriter('V7_PolesNodesTxSwitch.xlsx')
# dfNPTS.to_excel(MasterFile, 'Sheet1')
# MasterFile.save()
# print(dfNPTS['POLE_ID'].nunique()) #7237

Pole attachments analysis completed


In [95]:
#*****************************************************************************************************
# End for now
#*****************************************************************************************************
